In [1]:
import json
import os

from ml_tools import model_eval, nlp_prep, eda

# Found dataset of NYT Facebook posts

From https://data.world/martinchek/2012-2016-facebook-posts

In [109]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [110]:
# Sublime Text told me the encoding was UTF-16 LE with BOM
df = pd.read_csv('data/the-new-york-times-5281959998.csv', encoding='utf_16_le', 
                 engine='python')
df.head()

,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,shares_count,love_count,wow_count,haha_count,sad_count,thankful_count,angry_count,link,picture,posted_at
0,5281959998_416259568436280,5281959998,Did Barack Obama Save Ohio?,"'There are several pivotal states — Ohio, Flor...",Why the battle to take credit for Ohio’s ever-...,nytimes.com,link,shared_story,413,44,68,0,0,0,0,0,0,http://www.nytimes.com/2012/09/09/magazine/ohi...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-09-08 15:16:55
1,5281959998_268320586619940,5281959998,The Weatherman Is Not a Moron,"In 1940, the chance of an American being kille...",In the hocus-pocus realm of predicting the fut...,nytimes.com,link,shared_story,338,14,78,0,0,0,0,0,0,http://www.nytimes.com/2012/09/09/magazine/the...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-09-08 16:01:13
2,5281959998_467433813289503,5281959998,The Organic Fable,"'Organic has long since become an ideology, th...",A study exposes the hype behind a pseudoscient...,nytimes.com,link,shared_story,812,290,421,0,0,0,0,0,0,http://www.nytimes.com/2012/09/07/opinion/roge...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-09-08 16:44:02
3,5281959998_10150252974589999,5281959998,Timeline Photos,Our special Arts & Leisure section 'The New Se...,NaN,NaN,photo,added_photos,513,17,99,0,0,0,0,0,0,https://www.facebook.com/nytimes/photos/a.2835...,https://scontent.xx.fbcdn.net/v/t1.0-0/s130x13...,2012-09-08 17:32:18
4,5281959998_463550620334374,5281959998,Pot for Parents,'When I’m rolling around the floor with my gig...,"I am a more loving, attentive and patient fath...",nytimes.com,link,shared_story,502,83,170,0,0,0,0,0,0,http://www.nytimes.com/2012/09/08/opinion/how-...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-09-08 19:14:51


In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47867 entries, 0 to 47866
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              47867 non-null  object
 1   page_id         47867 non-null  int64 
 2   name            46574 non-null  object
 3   message         46132 non-null  object
 4   description     43512 non-null  object
 5   caption         42892 non-null  object
 6   post_type       47867 non-null  object
 7   status_type     47856 non-null  object
 8   likes_count     47867 non-null  int64 
 9   comments_count  47867 non-null  int64 
 10  shares_count    47867 non-null  int64 
 11  love_count      47867 non-null  int64 
 12  wow_count       47867 non-null  int64 
 13  haha_count      47867 non-null  int64 
 14  sad_count       47867 non-null  int64 
 15  thankful_count  47867 non-null  int64 
 16  angry_count     47867 non-null  int64 
 17  link            47201 non-null  object
 18  pictur

In [112]:
df.describe()

,page_id,likes_count,comments_count,shares_count,love_count,wow_count,haha_count,sad_count,thankful_count,angry_count
count,4.786700e+04,47867.000000,47867.000000,47867.000000,47867.000000,47867.000000,47867.000000,47867.000000,47867.000000,47867.000000
mean,5.281960e+09,1883.853761,179.163599,503.113690,38.096288,14.206301,15.359851,28.534084,0.032653,28.756429
std,0.000000e+00,6223.729785,998.989321,3258.821754,452.819483,151.503261,151.798665,318.864824,1.792674,357.463338
min,5.281960e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.281960e+09,335.000000,33.000000,57.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.281960e+09,697.000000,73.000000,147.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.281960e+09,1620.000000,173.000000,404.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,5.281960e+09,497519.000000,169805.000000,528856.000000,56070.000000,21121.000000,13214.000000,32316.000000,255.000000,48578.000000


# Gather Initial Article Info via NYT API

In [113]:
rerun_api = False

In [114]:
import requests
import time
import pickle
import gzip
import re

In [115]:
user_home = os.path.expanduser('~')
nyt_token = json.load(open(f'{user_home}/.secret/nytimes_api.json'))

In [116]:
years = [2012, 2013, 2014, 2015, 2016]
months = list(np.arange(1, 13, 1))
months

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

Gather initial basic article info for all articles between 2012 and 2016.

In [117]:
if rerun_api:

    base_url = 'https://api.nytimes.com/svc/archive/v1'
    total_count = 0
    articles = []

    for year in years:
        for month in months:
            url = f"{base_url}/{year}/{month}.json?api-key={nyt_token['api_key']}"
            response = requests.get(url)
            resp_dict=json.loads(response.text)

            article_count = resp_dict['response']['meta']['hits']
            print(f"Returned {article_count} articles from {month}, {year}")
            total_count += article_count

            articles.extend(resp_dict['response']['docs'])

            time.sleep(5)
        

In [118]:
# Export all article data to a file (list of dictionaries)
if rerun_api:
    f = gzip.open('nyt_articles_2012_to_2016.pickle.gz','wb')
    pickle.dump(articles, f)
    f.close()
else:
    # load article data from file if not gathering it fresh
    f = gzip.open('data/nyt_articles_2012_to_2016.pickle.gz','rb')
    articles = pickle.load(f)
    f.close()

In [119]:
# Create a dataframe from the NYT articles
df_nyt = pd.DataFrame(articles, columns=articles[0].keys())
df_nyt.head()

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,headline,keywords,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri
0,The best New Year’s Day news for Chicago’s spo...,https://www.nytimes.com/2012/01/01/sports/look...,The best New Year’s Day news for Chicago’s spo...,Dan McGrath writes a column for The Chicago Ne...,A,25B,The New York Times,"[{'rank': 0, 'subtype': 'jumbo', 'caption': No...",{'main': 'A Year Full of Missteps Is Finally O...,"[{'name': 'organizations', 'value': 'Chicago C...",2012-01-01T00:00:28+0000,article,National,Sports,"{'original': 'By Dan McGrath', 'person': [{'fi...",News,nyt://article/c304180e-4ab6-5ddb-89dc-8262c0fc...,912,nyt://article/c304180e-4ab6-5ddb-89dc-8262c0fc...
1,The man was preparing to board a flight in Mid...,https://www.nytimes.com/2012/01/01/us/servicem...,The man was preparing to board a flight in Mid...,HOUSTON — A man preparing to board a flight at...,A,19,The New York Times,[],{'main': 'Serviceman Held for Carrying Explosi...,"[{'name': 'subject', 'value': 'Airport Securit...",2012-01-01T00:09:12+0000,article,National,U.S.,"{'original': 'By Manny Fernandez', 'person': [...",News,nyt://article/1d442ebc-7050-5639-98a1-7af4dc9a...,242,nyt://article/1d442ebc-7050-5639-98a1-7af4dc9a...
2,"Setting the scene, Hollywood style, for what c...",https://fifthdown.blogs.nytimes.com/2011/12/31...,"Setting the scene, Hollywood style, for what c...","Yes, the Jets are a long shot to qualify for t...",NaN,NaN,The New York Times,[],{'main': 'Picturing How Ryan Brothers Could Se...,"[{'name': 'subject', 'value': 'Football', 'ran...",2012-01-01T00:14:59+0000,article,,Sports,"{'original': 'By George Bretherton', 'person':...",News,nyt://article/fc3e4b3f-e9ef-50b5-aaeb-0f8da040...,491,nyt://article/fc3e4b3f-e9ef-50b5-aaeb-0f8da040...
3,Experts said the broadcasts were most likely p...,https://thecaucus.blogs.nytimes.com/2011/12/31...,Experts said the broadcasts were most likely p...,DES MOINES — Can they do that?,NaN,NaN,The New York Times,[],{'main': 'Experts Weigh In on Outside Groups’ ...,"[{'name': 'subject', 'value': 'Campaign Financ...",2012-01-01T00:15:52+0000,article,,U.S.,"{'original': 'By Nicholas Confessore', 'person...",News,nyt://article/1bb54d6f-e9c3-5415-a754-cc0a0b4d...,449,nyt://article/1bb54d6f-e9c3-5415-a754-cc0a0b4d...
4,After more than a decade in which San Francisc...,https://www.nytimes.com/2012/01/01/us/an-unlik...,After more than a decade in which San Francisc...,"Susan Beckstead stepped out of her sky-blue, t...",A,25A,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'An Unlikely Group Rebels Against Pre...,"[{'name': 'subject', 'value': 'Historic Buildi...",2012-01-01T00:15:54+0000,article,National,U.S.,"{'original': 'By Matt Smith', 'person': [{'fir...",News,nyt://article/d22bc19c-f690-514c-bb55-d78a9928...,1164,nyt://article/d22bc19c-f690-514c-bb55-d78a9928...


In [120]:
df_nyt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425931 entries, 0 to 425930
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   abstract          425931 non-null  object
 1   web_url           425931 non-null  object
 2   snippet           425931 non-null  object
 3   lead_paragraph    425931 non-null  object
 4   print_section     243058 non-null  object
 5   print_page        243071 non-null  object
 6   source            425931 non-null  object
 7   multimedia        425931 non-null  object
 8   headline          425931 non-null  object
 9   keywords          425931 non-null  object
 10  pub_date          425931 non-null  object
 11  document_type     425931 non-null  object
 12  news_desk         425931 non-null  object
 13  section_name      425931 non-null  object
 14  byline            425931 non-null  object
 15  type_of_material  425931 non-null  object
 16  _id               425931 non-null  obj

# Match facebook posts to NYT articles

I learned that I can find the original facebook post by appending the second part of the `id` value after the underscore to this base path: `https://www.facebook.com/nytimes/posts/`.

## Link Expansion

I initially attempted to match Facebook posts to NYT articles by joining the Facebook post link on NYT article web_url.

However, I soon found that many of the Facebook links were shortened versions, so a direct comparison was not possible. 

Upon reviewing Facebook messages without anything populated int he link fie,d which are mostly photos and videos, I also determined that there were sometimes shortened links in the message body.

Because joining on direct link provides the most accurate match between posts and articles, I went back and did some processing to pull the links out of message bodies, and to expand the shortened versions of the Facebook links using urllib.

This process was split across multiple notebooks originally, since expanding the links was a time consuming process.

In [132]:
reexpand_links = False

### Pull links from message bodies

In [122]:
# mark Facebook posts that have a link in the link column
df.loc[(df['link'].isna()==False) & (
       (df['link'].str.contains('nytimes.com')) |
       (df['link'].str.contains('nyti.ms'))), 'has_nyt_link'] = 1

In [123]:
# review come examples of posts that don't have links 
df.loc[(df['link'].isna()==True) |
       (df['has_nyt_link'].isna()), ['post_type', 'link', 'message']][:5].values

array([['photo',
        'https://www.facebook.com/nytimes/photos/a.283559809998.33779.5281959998/10150252974559999/?type=3',
        "Our special Arts & Leisure section 'The New Season' is now online. Take a look at the upcoming offerings in arts and entertainment below. Are you looking forward to any particular events?Art: http://nyti.ms/TAYvoJClassical: http://nyti.ms/P41kvVDance: http://nyti.ms/Rx3YIhMovies: http://nyti.ms/Ow4dodPop: http://nyti.ms/RoEIJUTelevision: http://nyti.ms/Rx4fuRTheater: http://nyti.ms/OhVSZA(Credit: Jonathan Bartlett)"],
       ['photo',
        'https://www.facebook.com/nytimes/photos/a.283559809998.33779.5281959998/10150253031054999/?type=3',
        'During a surprise visit to Gators Dockside bar in Orlando, Florida, President Obama was introduced to a boy who was born in Hawaii. The president lit up and flashed the Hawaiian shaka hand sign, which the boy — Andre Wupperman, who will be 7 next week — returned. “You have a birth certificate?” the presiden

Looking at the messages, I see many of the messages do have links. Some also have more than one. These are what I want to pull out.

In [124]:
# create separate dataframe of just the posts without links but that
# also have something populated in the message body
df_messages = df.loc[((df['link'].isna()) | (df['has_nyt_link'].isna())) 
                     & (df['message'].isna()==False), 
                     ['id', 'post_type', 'message', 'description']]

In [125]:
# pull shortened links out into `message_link1`
df_messages = nlp_prep.pattern_match_in_df(df_messages, doc_col='message', 
                hit_col='message_link1', 
                pattern=r"(http[s]?://nyti.ms/[A-Za-z0-9/.]+)", 
                out_type='list', replace=False)

In [126]:
# pull full-length links out, if there are any, into `message_link2`
df_messages = nlp_prep.pattern_match_in_df(df_messages, doc_col='message', 
                hit_col='message_link2', 
                pattern=r"(http[s]?://nytimes/[A-Za-z0-9/.-_]+)", 
                out_type='list', replace=False)

In [128]:
# take a look at examples of posts where I pulled at least one shortened
# link out
df_messages.loc[df_messages['message_link1'].str.len() > 0]

,id,post_type,message,description,message_link1,message_link2
3,5281959998_10150252974589999,photo,Our special Arts & Leisure section 'The New Se...,NaN,"[http://nyti.ms/Ow4dodPop, http://nyti.ms/TAYv...",[]
5,5281959998_10150253031069999,photo,During a surprise visit to Gators Dockside bar...,NaN,[http://nyti.ms/NTb67a],[]
13,5281959998_10150253189039999,photo,President Obama got a huge lift—literally — fr...,NaN,[http://nyti.ms/PUEe9T],[]
14,5281959998_10150253207659999,photo,Serena Williams defeated Victoria Azarenka to ...,NaN,[http://nyti.ms/P8LIFN],[]
22,5281959998_10150253392344999,photo,What's on your menu for lunch or dinner today?...,NaN,[http://nyti.ms/RwXkrk],[]
...,...,...,...,...,...,...
47787,5281959998_1301608653216932,video,If you haven’t yet figured out a go-to recipe ...,NaN,[http://nyti.ms/2fpKB8d],[]
47792,5281959998_10150956594114999,photo,Ronald Reagan ended up winning 49 states in a ...,NaN,[http://nyti.ms/2fIAhNx],[]
47800,5281959998_10150956824229999,photo,"Read the letter that James Comey, the FBI dire...",NaN,[http://nyti.ms/2fpV8Se],[]
47815,5281959998_10150957229929999,photo,Election week is here. Enjoy unlimited access ...,NaN,[http://nyti.ms/2fPBG4g],[]


In [129]:
# take a look at examples of posts where I pulled a long link out. Are there any?
df_messages.loc[df_messages['message_link2'].str.len() > 0]

,id,post_type,message,description,message_link1,message_link2


OK so only link1 (the shortened version) got populated; the messages didn't ever include full nytimes links. Let's pull out the first shortened link from each post rather than try to get multiple.

In [130]:
# Pull first shortened link from list of links
df_messages['message_link'] = df_messages['message_link1'].map(
    lambda x: x[0] if len(x) > 0 else '')

In [131]:
# review links pulled out
df_messages.loc[df_messages['message_link'] != '']

,id,post_type,message,description,message_link1,message_link2,message_link
3,5281959998_10150252974589999,photo,Our special Arts & Leisure section 'The New Se...,NaN,"[http://nyti.ms/Ow4dodPop, http://nyti.ms/TAYv...",[],http://nyti.ms/Ow4dodPop
5,5281959998_10150253031069999,photo,During a surprise visit to Gators Dockside bar...,NaN,[http://nyti.ms/NTb67a],[],http://nyti.ms/NTb67a
13,5281959998_10150253189039999,photo,President Obama got a huge lift—literally — fr...,NaN,[http://nyti.ms/PUEe9T],[],http://nyti.ms/PUEe9T
14,5281959998_10150253207659999,photo,Serena Williams defeated Victoria Azarenka to ...,NaN,[http://nyti.ms/P8LIFN],[],http://nyti.ms/P8LIFN
22,5281959998_10150253392344999,photo,What's on your menu for lunch or dinner today?...,NaN,[http://nyti.ms/RwXkrk],[],http://nyti.ms/RwXkrk
...,...,...,...,...,...,...,...
47787,5281959998_1301608653216932,video,If you haven’t yet figured out a go-to recipe ...,NaN,[http://nyti.ms/2fpKB8d],[],http://nyti.ms/2fpKB8d
47792,5281959998_10150956594114999,photo,Ronald Reagan ended up winning 49 states in a ...,NaN,[http://nyti.ms/2fIAhNx],[],http://nyti.ms/2fIAhNx
47800,5281959998_10150956824229999,photo,"Read the letter that James Comey, the FBI dire...",NaN,[http://nyti.ms/2fpV8Se],[],http://nyti.ms/2fpV8Se
47815,5281959998_10150957229929999,photo,Election week is here. Enjoy unlimited access ...,NaN,[http://nyti.ms/2fPBG4g],[],http://nyti.ms/2fPBG4g


### Expand links pulled from messages

In [54]:
if reexpand_links:

    # use urllib to expand URLs, since some are shortened
    # code adapted from:
    # https://stackoverflow.com/questions/4201062/how-can-i-unshorten-a-url
    expanded_link = []
    session = requests.Session()  # so connections are recycled
    count = 0

    for i in df_messages.loc[df_messages['message_link'] != ''].index:
        try:
            resp = session.head(df_messages.at[i, 'message_link'], 
                                allow_redirects=True)
            expanded_link.append([df_messages.at[i, 'id'], resp.url])
        except:
            print(f"Error at index {i}")

        count += 1

        if count % 500 == 0:
            print(f"{count} links processed")
            
            # Every 500 links, dump list to json
            with open(f'data/expanded_links_frommessages.json', 'w') as f:
                json.dump(expanded_link, f)
                f.close()
            
    # at the end, dump final list to json
    with open(f'data/expanded_links_frommessages.json', 'w') as f:
        json.dump(expanded_link, f)
        f.close()

Error at index 1989
Error at index 2266
Error at index 2913
500 links processed
1000 links processed
1500 links processed
2000 links processed
2500 links processed
3000 links processed


[['5281959998_10150252974589999', 'https://www.nytimes.com/'],
 ['5281959998_10150253031069999',
  'https://thecaucus.blogs.nytimes.com/2012/09/08/obama-makes-birth-certificate-joke/?smid=FB-nytimes&WT.mc_id=PO-E-FB-SM-PIX-OMB-090812-NYT-NA&WT.mc_ev=click'],
 ['5281959998_10150253189039999',
  'https://thecaucus.blogs.nytimes.com/2012/09/09/obama-gets-a-lift-on-the-trail/?smid=FB-nytimes&WT.mc_id=PO-E-FB-SM-PIX-OGL-090912-NYT-NA&WT.mc_ev=click'],
 ['5281959998_10150253207659999',
  'https://archive.nytimes.com/www.nytimes.com/interactive/2012/09/08/sports/tennis/usopen-womens-final-live.html?WT.mc_ev=click&WT.mc_id=SP-E-FB-SM-PIX-LUF-090912-NYT-NA&smid=FB-nytimes'],
 ['5281959998_10150253392344999',
  'https://well.blogs.nytimes.com/2012/09/07/wholesome-risotto-with-whole-grains/?smid=FB-nytimes&WT.mc_id=HL-E-FB-SM-PIX-WRW-091012-NYT-NA&WT.mc_ev=click']]

### Identify Shortened Post Links

In [133]:
# We have a link populated, and it's in the shortened version
df.loc[(df['link'].isna()==False) & 
       (df['link'].str.contains('nyti.ms', regex=False))]

,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,love_count,wow_count,haha_count,sad_count,thankful_count,angry_count,link,picture,posted_at,has_nyt_link
1834,5281959998_298385756956423,5281959998,"‘Shoot One, Please’",Have you ever been hunting? How did you feel t...,This short documentary follows a 15-year-old f...,nytimes.com,link,shared_story,85,88,...,0,0,0,0,0,0,http://nyti.ms/YduMRx,https://external.xx.fbcdn.net/safe_image.php?d...,2013-03-06 23:15:54,1.0
1837,5281959998_338711676230154,5281959998,"Face-Lift at Facebook, to Keep Its Users Engaged","The redesign of the News Feed, which Facebook ...",Facebook plans to announce a substantial redes...,nytimes.com,link,shared_story,325,122,...,0,0,0,0,0,0,http://nyti.ms/WwFEjm,https://external.xx.fbcdn.net/safe_image.php?d...,2013-03-07 15:15:36,1.0
1842,5281959998_256954574440999,5281959998,"The Good, Racist People",What does it mean that last month the actor Fo...,What does it mean when Forest Whitaker is fris...,nytimes.com,link,shared_story,1021,160,...,0,0,0,0,0,0,http://nyti.ms/ZiJ0Sb,https://external.xx.fbcdn.net/safe_image.php?d...,2013-03-07 19:00:14,1.0
1844,5281959998_280841352048945,5281959998,We’re Through. Check the App.,The pain of a breakup is nourished by an endle...,Using a smartphone or your computer to get rid...,nytimes.com,link,shared_story,333,65,...,0,0,0,0,0,0,http://nyti.ms/W8dSHj,https://external.xx.fbcdn.net/safe_image.php?d...,2013-03-07 20:45:04,1.0
1851,5281959998_519969881380141,5281959998,Bin Laden Relative Pleads Not Guilty in Terror...,In a courtroom just blocks from the site of th...,In a courtroom just blocks from the site of th...,nytimes.com,link,shared_story,94,14,...,0,0,0,0,0,0,http://nyti.ms/XvsHF2,https://external.xx.fbcdn.net/safe_image.php?d...,2013-03-08 16:11:35,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47628,5281959998_10150951110934999,5281959998,"Safer to Puff, E-Cigarettes Can’t Shake Their ...",Evidence suggests vaping is far less dangerous...,A growing number of experts say that the portr...,nytimes.com,link,shared_story,500,172,...,12,9,18,1,0,2,http://nyti.ms/2fvvZck,https://external.xx.fbcdn.net/safe_image.php?d...,2016-11-03 04:03:14,1.0
47635,5281959998_10150951251454999,5281959998,Farmers’ Unchecked Crop Burning Fuels India’s ...,India is desperate to reduce air pollution in ...,The burning of straw is a major contributor to...,nytimes.com,link,shared_story,264,31,...,2,11,3,42,0,22,http://nyti.ms/2e2JGhI,https://external.xx.fbcdn.net/safe_image.php?d...,2016-11-03 08:55:00,1.0
47706,5281959998_10150953813204999,5281959998,Ex-Christie Aides Convicted in George Washingt...,Breaking News: Two former aides to Governor Ch...,"The two defendants, Bridget Anne Kelly and Bil...",nytimes.com,link,shared_story,4368,391,...,98,356,436,20,0,52,http://nyti.ms/2f8PjIa,https://external.xx.fbcdn.net/safe_image.php?d...,2016-11-04 15:37:35,1.0
47721,5281959998_10150954183094999,5281959998,Immigrants Living ‘On Pause’ Fear Obama’s Leni...,"At the end of the day, her fear returns: When ...",Two families who came to the United States ill...,nytimes.com,link,shared_story,298,147,...,2,17,5,342,0,14,http://nyti.ms/2ewyZ5G,https://external.xx.fbcdn.net/safe_image.php?d...,2016-11-04 21:55:00,1.0


### Expand shortened original links

In [99]:
if reexpand_links:
    
    # use urllib to expand URLs, since some are shortened
    # code adapted from:
    # https://stackoverflow.com/questions/4201062/how-can-i-unshorten-a-url
    expanded_link = []
    session = requests.Session()  # so connections are recycled
    count = 0

    for i in df.loc[(df['link'].isna()==False) & 
           (df['link'].str.contains('nyti.ms', regex=False))].index:

        try:
            resp = session.head(df.at[i, 'link'], allow_redirects=True)
            expanded_link.append([df.at[i, 'id'], resp.url])
        except:
            print(f"Error at index {i}")

        count += 1

        if count % 500 == 0:
            print(f"{count} links processed")
            with open(f'data/expanded_links.json', 'w') as f:
                json.dump(expanded_link, f)
                f.close()

    # final export:
    with open(f'data/expanded_links.json', 'w') as f:
        json.dump(expanded_link, f)
        f.close()


Error at index 6010
500 links processed
1000 links processed
1500 links processed
2000 links processed
2500 links processed
3000 links processed
3500 links processed
4000 links processed
4500 links processed
5000 links processed
5500 links processed
6000 links processed
6500 links processed
7000 links processed
7500 links processed
8000 links processed
8500 links processed
9000 links processed
9500 links processed
10000 links processed
10500 links processed
11000 links processed
11500 links processed
12000 links processed
12500 links processed
13000 links processed
13500 links processed
14000 links processed
14500 links processed
15000 links processed
15500 links processed
16000 links processed
16500 links processed
17000 links processed
17500 links processed
18000 links processed
18500 links processed
19000 links processed
19500 links processed
20000 links processed
20500 links processed
21000 links processed
21500 links processed
22000 links processed
22500 links processed
23000 link

[['5281959998_10150485819419999',
  'https://cooking.nytimes.com/recipes/8245-smoked-turkey'],
 ['5281959998_10150485829769999',
  'https://www.nytimes.com/2014/11/16/fashion/weddings/solange-knowles-the-younger-sister-of-beyonce-knowles-marries-music-video-director.html'],
 ['5281959998_10150485830749999',
  'https://www.nytimes.com/2014/11/17/us/groups-in-ferguson-prepare-for-grand-jury-decision.html?smid=fb-nytimes&smtyp=cur&bicmp=AD&bicmlukp=WT.mc_id&bicmst=1409232722000&bicmet=1419773522000'],
 ['5281959998_10150485840494999',
  'https://www.nytimes.com/2014/11/16/arts/television/one-writers-diary-for-television-pitch-season.html?smid=fb-nytimes&smtyp=cur&bicmp=AD&bicmlukp=WT.mc_id&bicmst=1409232722000&bicmet=1419773522000&_r=0'],
 ['5281959998_10150485850274999',
  'https://www.nytimes.com/2014/11/15/us/one-drug-or-2-parents-see-risk-but-also-hope.html?smid=fb-nytimes&smtyp=cur&bicmp=AD&bicmlukp=WT.mc_id&bicmst=1409232722000&bicmet=1419773522000']]

### Combine expanded links with main df

Initially, I ran the link expansion in several batches, exported the results to individual files, and combined them.

Here, I've saved the final result which we can import all at once.

In [153]:
# read in final results of expanding multiple sets of shortened links
df_expanded = pd.read_csv('data/expanded_links_all.csv.gz', index_col=0)
df_expanded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38119 entries, 0 to 38118
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             38119 non-null  object
 1   link_expanded  38119 non-null  object
dtypes: object(2)
memory usage: 893.4+ KB


In [154]:
df = df.merge(df_expanded, how='left', left_on='id', right_on='id',
                   suffixes=('', '_new'))
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47867 entries, 0 to 47866
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              47867 non-null  object 
 1   page_id         47867 non-null  int64  
 2   name            46574 non-null  object 
 3   message         46132 non-null  object 
 4   description     43512 non-null  object 
 5   caption         42892 non-null  object 
 6   post_type       47867 non-null  object 
 7   status_type     47856 non-null  object 
 8   likes_count     47867 non-null  int64  
 9   comments_count  47867 non-null  int64  
 10  shares_count    47867 non-null  int64  
 11  love_count      47867 non-null  int64  
 12  wow_count       47867 non-null  int64  
 13  haha_count      47867 non-null  int64  
 14  sad_count       47867 non-null  int64  
 15  thankful_count  47867 non-null  int64  
 16  angry_count     47867 non-null  int64  
 17  link            47201 non-null 

In [155]:
# backup original link field
df['link_backup'] = df['link']
df['link_backup'].head()

0    http://www.nytimes.com/2012/09/09/magazine/ohi...
1    http://www.nytimes.com/2012/09/09/magazine/the...
2    http://www.nytimes.com/2012/09/07/opinion/roge...
3    https://www.facebook.com/nytimes/photos/a.2835...
4    http://www.nytimes.com/2012/09/08/opinion/how-...
Name: link_backup, dtype: object

In [156]:
# Overwrite link column with expanded where we have an expanded
df.loc[df['link_expanded'].isna()==False, 'link'] = \
df.loc[df['link_expanded'].isna()==False, 'link_expanded']

In [161]:
df.loc[(df['link'].isna())]

,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,haha_count,sad_count,thankful_count,angry_count,link,picture,posted_at,has_nyt_link,link_expanded,link_backup
158,5281959998_10150255185794999,5281959998,NaN,We are currently working with Facebook to reso...,NaN,NaN,status,mobile_status_update,495,50,...,0,0,0,0,NaN,NaN,2012-09-21 19:01:04,NaN,NaN,NaN
960,5281959998_10150270884874999,5281959998,NaN,Speaking about the Connecticut school shooting...,NaN,NaN,status,mobile_status_update,3112,760,...,0,0,0,0,NaN,NaN,2012-12-14 19:13:14,NaN,NaN,NaN
1025,5281959998_10150271830399999,5281959998,NaN,"Good evening. In a few minutes, at 8 p.m. East...",NaN,NaN,status,mobile_status_update,702,71,...,0,0,0,0,NaN,NaN,2012-12-20 00:57:05,NaN,NaN,NaN
1026,5281959998_10150271831404999,5281959998,NaN,"Hi again. Jacques Steinberg here, senior edito...",NaN,NaN,status,mobile_status_update,330,111,...,0,0,0,0,NaN,NaN,2012-12-20 01:00:14,NaN,NaN,NaN
1476,5281959998_10150283660154999,5281959998,NaN,"We've got a ballgame. Baltimore, 31 - San Fran...",NaN,NaN,status,mobile_status_update,715,230,...,0,0,0,0,NaN,NaN,2013-02-04 03:12:26,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39181,5281959998_10150808970444999,5281959998,NaN,"For years, couples have vied for a place in ou...",NaN,NaN,status,mobile_status_update,23,3,...,0,0,0,0,NaN,NaN,2016-05-14 14:50:08,NaN,NaN,NaN
45072,5281959998_10150897137829999,5281959998,NaN,"During a heated campaign, important issues can...",NaN,NaN,status,mobile_status_update,90,443,...,9,0,0,4,NaN,NaN,2016-09-09 18:40:00,NaN,NaN,NaN
46198,5281959998_10150915570479999,5281959998,NaN,A Times photographer visited El Pampero Hospit...,NaN,NaN,status,mobile_status_update,125,35,...,0,100,0,21,NaN,NaN,2016-10-01 23:55:00,NaN,NaN,NaN
47094,5281959998_10150933823424999,5281959998,NaN,UPDATE: The New York Times website is back up....,NaN,NaN,status,mobile_status_update,224,76,...,10,10,0,4,NaN,NaN,2016-10-19 08:32:30,NaN,NaN,NaN


In [162]:
df.loc[(df['link'].isna()), 'post_type'].value_counts()

status    117
Name: post_type, dtype: int64

Now all of the entries without links are status updates, which makes sense that they may not have links associated with them.

In [165]:
# are there still any shortened links in the main link field?
df.loc[(df['link'].isna()==False) & 
       (df['link'].str.contains('nyti.ms', regex=False)), 'link'].values

array(['http://nyti.ms/1kFqgM3',
       'https://www.nytimes.com/2014/09/05/us/politics/chris-christie-on-mexico-tour-unplugs-the-amplifier.html?smid=fb-nytimes&smtyp=cur&bicmp=AD&bicmlukp=WT.mc_id&bicmst=1409232722000&bicmet=1419773522000http://nyti.ms/1qtmLKV',
       'https://www.nytimes.com/2014/09/06/upshot/jobs-report-not-good-but-not-terrible-either.html?smid=fb-nytimes&smtyp=cur&bicmp=AD&bicmlukp=WT.mc_id&bicmst=1409232722000&bicmet=1419773522000http://nyti.ms/1oMgNj1'],
      dtype=object)

The first is a link to submit something, which appears to not be live anymore.

The second and third just have that shortened pattern in the parameters. Great, all are now populated with the expanded links!

## Link Preprocessing

Facebook links start with `http` instead of `https`, so need to adjust that.

Facebook links also have parameters added to the end that don't exist in the NYT links, so I'm going to remove those.

In [166]:
# replace http with https, since https is used in nytimes links
df['trim_link'] = df['link'].map(lambda x: str.replace(x, 'http:', 'https:') if
                             type(x)==str else '')

In [167]:
# pull out the base page link, without any extra & and ?
df['trim_link'] = df['trim_link'].map(
    lambda x: re.search(r"^([^?&]+)", x).group(1) 
                if (type(x)==str and re.search(r"^([^?&]+)", x)) else '')

In [168]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47867 entries, 0 to 47866
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              47867 non-null  object 
 1   page_id         47867 non-null  int64  
 2   name            46574 non-null  object 
 3   message         46132 non-null  object 
 4   description     43512 non-null  object 
 5   caption         42892 non-null  object 
 6   post_type       47867 non-null  object 
 7   status_type     47856 non-null  object 
 8   likes_count     47867 non-null  int64  
 9   comments_count  47867 non-null  int64  
 10  shares_count    47867 non-null  int64  
 11  love_count      47867 non-null  int64  
 12  wow_count       47867 non-null  int64  
 13  haha_count      47867 non-null  int64  
 14  sad_count       47867 non-null  int64  
 15  thankful_count  47867 non-null  int64  
 16  angry_count     47867 non-null  int64  
 17  link            47750 non-null 

## Review Duplicates

In [172]:
# replace empty strings with nulls
df.loc[df['trim_link']=='', 'trim_link'] = np.nan

In [173]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47867 entries, 0 to 47866
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              47867 non-null  object 
 1   page_id         47867 non-null  int64  
 2   name            46574 non-null  object 
 3   message         46132 non-null  object 
 4   description     43512 non-null  object 
 5   caption         42892 non-null  object 
 6   post_type       47867 non-null  object 
 7   status_type     47856 non-null  object 
 8   likes_count     47867 non-null  int64  
 9   comments_count  47867 non-null  int64  
 10  shares_count    47867 non-null  int64  
 11  love_count      47867 non-null  int64  
 12  wow_count       47867 non-null  int64  
 13  haha_count      47867 non-null  int64  
 14  sad_count       47867 non-null  int64  
 15  thankful_count  47867 non-null  int64  
 16  angry_count     47867 non-null  int64  
 17  link            47750 non-null 

In [174]:
# Review duplicates where we have the same link, description, and status_type
df.loc[df.duplicated(subset=['description', 'trim_link', 'status_type'], 
                     keep=False)]

,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,sad_count,thankful_count,angry_count,link,picture,posted_at,has_nyt_link,link_expanded,link_backup,trim_link
27,5281959998_10150253426994999,5281959998,Timeline Photos,Follow live updates from the U.S. Open Men's F...,NaN,NaN,photo,added_photos,467,93,...,0,0,0,https://archive.nytimes.com/www.nytimes.com/in...,https://scontent.xx.fbcdn.net/v/t1.0-0/s130x13...,2012-09-10 21:28:56,NaN,https://archive.nytimes.com/www.nytimes.com/in...,https://www.facebook.com/nytimes/photos/a.2835...,https://archive.nytimes.com/www.nytimes.com/in...
29,5281959998_10150253458449999,5281959998,Timeline Photos,Andy Murray defeated Novak Djokovic to win the...,NaN,NaN,photo,added_photos,4324,193,...,0,0,0,https://archive.nytimes.com/www.nytimes.com/in...,https://scontent.xx.fbcdn.net/v/t1.0-0/s130x13...,2012-09-11 01:04:31,NaN,https://archive.nytimes.com/www.nytimes.com/in...,https://www.facebook.com/nytimes/photos/a.2835...,https://archive.nytimes.com/www.nytimes.com/in...
158,5281959998_10150255185794999,5281959998,NaN,We are currently working with Facebook to reso...,NaN,NaN,status,mobile_status_update,495,50,...,0,0,0,NaN,NaN,2012-09-21 19:01:04,NaN,NaN,NaN,NaN
218,5281959998_10150256227144999,5281959998,NaN,N.F.L. and Referees Union Agree on Deal to End...,NaN,NaN,status,mobile_status_update,933,48,...,0,0,0,https://www.nytimes.com/?smid=FB-nytimes&WT.mc...,NaN,2012-09-27 04:02:02,NaN,https://www.nytimes.com/?smid=FB-nytimes&WT.mc...,NaN,https://www.nytimes.com/
277,5281959998_480453791987573,5281959998,"Unfriending Someone, Before Facebook","When it's time for a friendship to end, is it ...","How much better things were 50 years ago, when...",nytimes.com,link,shared_story,454,41,...,0,0,0,http://www.nytimes.com/2012/10/03/booming/03wa...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-10-03 16:10:40,1.0,NaN,http://www.nytimes.com/2012/10/03/booming/03wa...,https://www.nytimes.com/2012/10/03/booming/03w...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47762,5281959998_10150950200729999,5281959998,Man vs. Marathon,Tens of thousands of runners will tackle the T...,One scientist’s quixotic quest to propel a run...,nytimes.com,link,shared_story,185,19,...,0,0,1,http://www.nytimes.com/2016/05/15/sports/two-h...,https://external.xx.fbcdn.net/safe_image.php?d...,2016-11-06 00:55:01,1.0,NaN,http://www.nytimes.com/2016/05/15/sports/two-h...,https://www.nytimes.com/2016/05/15/sports/two-...
47775,5281959998_10150951035014999,5281959998,The Governing Cancer of Our Time,'People say that Trump is an unconventional ca...,Donald Trump’s candidacy is the culmination of...,nytimes.com,link,shared_story,16005,982,...,622,0,99,http://www.nytimes.com/2016/02/26/opinion/the-...,https://external.xx.fbcdn.net/safe_image.php?d...,2016-11-06 11:55:00,1.0,NaN,http://www.nytimes.com/2016/02/26/opinion/the-...,https://www.nytimes.com/2016/02/26/opinion/the...
47801,5281959998_10150956854609999,5281959998,Latest Election Polls 2016,"The latest U.S. and swing state polls, updated...",Get the latest national and state polls on the...,nytimes.com,link,shared_story,4429,462,...,38,0,60,http://www.nytimes.com/interactive/2016/us/ele...,https://external.xx.fbcdn.net/safe_image.php?d...,2016-11-06 21:45:46,1.0,NaN,http://www.nytimes.com/interactive/2016/us/ele...,https://www.nytimes.com/interactive/2016/us/el...
47806,5281959998_10150953848459999,5281959998,Pasta e Fagioli Recipe,A cozy classic. (via The New York Times Food),This is a classic Italian bean and pasta soup ...,cooking.nytimes.com,link,shared_story,587,54,...,0,0,2,http://cooking.nytimes.com/recipes/1016031-pas...,https://external.xx.fbcdn.net/safe_image.php?d...,2016-11-06 23:55:00,1.0,NaN,http://cooking.nytimes.com/recipes/1016031-pas...,https://cooking.nytimes.com/recipes/1016031-pa...


In [175]:
df.loc[[47641, 47704]]

,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,sad_count,thankful_count,angry_count,link,picture,posted_at,has_nyt_link,link_expanded,link_backup,trim_link
47641,5281959998_10150952237769999,5281959998,"Hillary Clinton Still Leads a Tighter Race, Ti...",Breaking News: Hillary Clinton holds a 3-point...,Most voters say that their minds are made up a...,nytimes.com,link,shared_story,6102,634,...,9,0,28,http://www.nytimes.com/2016/11/04/us/politics/...,https://external.xx.fbcdn.net/safe_image.php?d...,2016-11-03 11:03:02,1.0,NaN,http://www.nytimes.com/2016/11/04/us/politics/...,https://www.nytimes.com/2016/11/04/us/politics...
47704,5281959998_10150953685229999,5281959998,Voters Express Disgust Over U.S. Politics in N...,A majority of voters say that neither Hillary ...,Most voters say that their minds are made up a...,nytimes.com,link,shared_story,474,182,...,52,0,4,http://www.nytimes.com/2016/11/04/us/politics/...,https://external.xx.fbcdn.net/safe_image.php?d...,2016-11-04 14:55:00,1.0,NaN,http://www.nytimes.com/2016/11/04/us/politics/...,https://www.nytimes.com/2016/11/04/us/politics...


In [176]:
ex_desc = df.loc[[47641]].values
ex_desc

array([['5281959998_10150952237769999', 5281959998,
        'Hillary Clinton Still Leads a Tighter Race, Times/CBS News Poll Shows',
        'Breaking News: Hillary Clinton holds a 3-point lead over Donald J. Trump in the latest Times/CBS News poll. Most voters say their minds are made up.',
        'Most voters say that their minds are made up and that late revelations about Mrs. Clinton and Donald J. Trump made no significant difference to them.',
        'nytimes.com', 'link', 'shared_story', 6102, 634, 628, 715, 31,
        82, 9, 0, 28,
        'http://www.nytimes.com/2016/11/04/us/politics/hillary-clinton-donald-trump-poll.html?smid=fb-nytimes&smtyp=cur',
        'https://external.xx.fbcdn.net/safe_image.php?d=AQBMxkLxy577UjpI&w=130&h=130&url=https%3A%2F%2Fstatic01.nyt.com%2Fimages%2F2016%2F11%2F03%2Fus%2F04poll1%2F04poll1-facebookJumbo.jpg&cfs=1&sx=224&sy=0&sw=550&sh=550',
        '2016-11-03 11:03:02', 1.0, nan,
        'http://www.nytimes.com/2016/11/04/us/politics/hillary-cli

In [177]:
df_nyt.loc[[416125]].values

array([['Most voters say that their minds are made up and that late revelations about Mrs. Clinton and Donald J. Trump made no significant difference to them.',
        'https://www.nytimes.com/2016/11/04/us/politics/hillary-clinton-donald-trump-poll.html',
        'Most voters say that their minds are made up and that late revelations about Mrs. Clinton and Donald J. Trump made no significant difference to them.',
        'An overwhelming majority of voters are disgusted by the state of American politics, and many harbor doubts that either major-party nominee can unite the country after a historically ugly presidential campaign, according to the final pre-election New York Times/CBS News Poll.',
        'A', '1', 'The New York Times',
        list([{'rank': 0, 'subtype': 'thumbnail', 'caption': None, 'credit': None, 'type': 'image', 'url': 'images/2016/11/03/us/04poll1/04poll1-thumbStandard.jpg', 'height': 75, 'width': 75, 'subType': 'thumbnail', 'crop_name': 'thumbStandard', 'legacy'

In [178]:
df.loc[[47641, 47704]].values

array([['5281959998_10150952237769999', 5281959998,
        'Hillary Clinton Still Leads a Tighter Race, Times/CBS News Poll Shows',
        'Breaking News: Hillary Clinton holds a 3-point lead over Donald J. Trump in the latest Times/CBS News poll. Most voters say their minds are made up.',
        'Most voters say that their minds are made up and that late revelations about Mrs. Clinton and Donald J. Trump made no significant difference to them.',
        'nytimes.com', 'link', 'shared_story', 6102, 634, 628, 715, 31,
        82, 9, 0, 28,
        'http://www.nytimes.com/2016/11/04/us/politics/hillary-clinton-donald-trump-poll.html?smid=fb-nytimes&smtyp=cur',
        'https://external.xx.fbcdn.net/safe_image.php?d=AQBMxkLxy577UjpI&w=130&h=130&url=https%3A%2F%2Fstatic01.nyt.com%2Fimages%2F2016%2F11%2F03%2Fus%2F04poll1%2F04poll1-facebookJumbo.jpg&cfs=1&sx=224&sy=0&sw=550&sh=550',
        '2016-11-03 11:03:02', 1.0, nan,
        'http://www.nytimes.com/2016/11/04/us/politics/hillary-cli

I'm not sure I can just drop these completely... It looks like maybe the duplicates could represent some experimentation with framing the story differently to capture the attention of audiences better, and it's possible this resulted in significantly different numbers of comments and likes. It's also possible that the change is due to going with an initial headline for breaking news, and changing it later. The first example was a breaking news story at first, and it was also related to results of a poll, from which many conclusions could be drawn. Maybe they just needed more time to tease additional insights out of the story.

What I would like to do is mark these duplicates so I can try to match the API articles based on just one copy, and then I will need to go back and take a look at the details to decide which version should actually be kept. 

Perhaps both, since they will have different message values and different numbers of comments and likes, so we could do a direct comparison to see which did better. Maybe the second post could be in the test set?

In [183]:
df.loc[(df.duplicated(subset=['trim_link', 'status_type'], keep=False)) &
       (df['trim_link'].isna()==False), 
       ['trim_link', 'status_type', 'posted_at']][:10].values

array([['https://www.nytimes.com/2012/09/11/education/teacher-strike-begins-in-chicago-amid-signs-that-deal-isnt-close.html',
        'shared_story', '2012-09-10 15:36:47'],
       ['https://archive.nytimes.com/www.nytimes.com/interactive/2012/09/10/sports/tennis/usopen-mens-final-live.html',
        'added_photos', '2012-09-10 21:28:56'],
       ['https://archive.nytimes.com/www.nytimes.com/interactive/2012/09/10/sports/tennis/usopen-mens-final-live.html',
        'added_photos', '2012-09-11 01:04:31'],
       ['https://www.nytimes.com/2012/09/11/education/teacher-strike-begins-in-chicago-amid-signs-that-deal-isnt-close.html',
        'shared_story', '2012-09-11 12:05:55'],
       ['https://www.nytimes.com/2012/09/13/world/middleeast/us-envoy-to-libya-is-reported-killed.html',
        'shared_story', '2012-09-12 12:53:53'],
       ['https://www.nytimes.com/2012/09/13/world/middleeast/us-envoy-to-libya-is-reported-killed.html',
        'shared_story', '2012-09-12 16:10:11'],
       ['h

There are duplicate links and post types, but they look to have been posted at different times, which means links to the same articles were posted multiple times.

In [184]:
# populate 'dupes_on' column for all copies indicating how duplicate status
# was determined. `link` means these are duplicates based on the link
# specifically don't include duplicates
df.loc[(df.duplicated(subset=['trim_link', 'status_type'], keep=False)) &
       (df['trim_link'].isna()==False), 'dupes_on_link'] = 1

# populate `dupe_child` column for all but the first occurences
df.loc[(df.duplicated(subset=['trim_link', 'status_type'], keep='first'))
       & (df['trim_link'].isna()==False), 'dupe_link_child'] = 1

In [185]:
# identify duplicates based on description only, since I want to try to
# use that to join with nyt articles
df.loc[(df.duplicated(subset=['description'], keep=False)) &
        (df['description'].isna()==False), 'dupes_on_desc'] = 1

df.loc[(df.duplicated(subset=['description'], keep='first')) &
       (df['description'].isna()==False), 'dupe_desc_child'] = 1

In [186]:
# identify duplicates based on name
df.loc[(df.duplicated(subset=['name'], keep=False)) &
        (df['name'].isna()==False), 'dupes_on_name'] = 1

df.loc[(df.duplicated(subset=['name'], keep='first')) &
       (df['name'].isna()==False), 'dupe_name_child'] = 1

In [187]:
df.loc[(df['dupes_on_link']==1) | 
       (df['dupes_on_desc']==1) | 
       (df['dupes_on_name']==1)]

,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,has_nyt_link,link_expanded,link_backup,trim_link,dupes_on_link,dupe_link_child,dupes_on_desc,dupe_desc_child,dupes_on_name,dupe_name_child
3,5281959998_10150252974589999,5281959998,Timeline Photos,Our special Arts & Leisure section 'The New Se...,NaN,NaN,photo,added_photos,513,17,...,NaN,NaN,https://www.facebook.com/nytimes/photos/a.2835...,https://www.facebook.com/nytimes/photos/a.2835...,NaN,NaN,NaN,NaN,1.0,NaN
5,5281959998_10150253031069999,5281959998,Timeline Photos,During a surprise visit to Gators Dockside bar...,NaN,NaN,photo,added_photos,9012,594,...,NaN,https://thecaucus.blogs.nytimes.com/2012/09/08...,https://www.facebook.com/nytimes/photos/a.2835...,https://thecaucus.blogs.nytimes.com/2012/09/08...,NaN,NaN,NaN,NaN,1.0,1.0
13,5281959998_10150253189039999,5281959998,Timeline Photos,President Obama got a huge lift—literally — fr...,NaN,NaN,photo,added_photos,16918,1415,...,NaN,https://thecaucus.blogs.nytimes.com/2012/09/09...,https://www.facebook.com/nytimes/photos/a.2835...,https://thecaucus.blogs.nytimes.com/2012/09/09...,NaN,NaN,NaN,NaN,1.0,1.0
14,5281959998_10150253207659999,5281959998,Timeline Photos,Serena Williams defeated Victoria Azarenka to ...,NaN,NaN,photo,added_photos,7629,353,...,NaN,https://archive.nytimes.com/www.nytimes.com/in...,https://www.facebook.com/nytimes/photos/a.2835...,https://archive.nytimes.com/www.nytimes.com/in...,NaN,NaN,NaN,NaN,1.0,1.0
20,5281959998_353360298082368,5281959998,"Teacher Strike Begins in Chicago, Amid Signs T...","Teachers in Chicago Public Schools, the nation...","As about 350,000 students got an unexpected da...",nytimes.com,link,shared_story,187,19,...,1.0,NaN,http://www.nytimes.com/2012/09/11/education/te...,https://www.nytimes.com/2012/09/11/education/t...,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47840,5281959998_10150957971334999,5281959998,The New York Times - Theater,"We're with Mike Daisey, who is performing a po...","Learn about the making of Donald J. Trump, as ...",NaN,video,shared_story,229,59,...,NaN,NaN,https://www.facebook.com/nytimestheater/videos...,https://www.facebook.com/nytimestheater/videos...,NaN,NaN,NaN,NaN,1.0,1.0
47845,5281959998_10150958061204999,5281959998,Nicholas Kristof,"Nicholas Kristof, a columnist for The New York...",Our last Facebook Live before the election! Ch...,NaN,video,shared_story,144,40,...,NaN,NaN,https://www.facebook.com/kristof/videos/101545...,https://www.facebook.com/kristof/videos/101545...,NaN,NaN,NaN,NaN,1.0,1.0
47847,5281959998_10150958085764999,5281959998,The New York Times - Well - Health,Lauren Hard and Paul Moon of The New York Time...,"We're at Pure Yoga with Sarah Attia, founder o...",NaN,video,shared_story,70,6,...,NaN,NaN,https://www.facebook.com/WellNYT/videos/116561...,https://www.facebook.com/WellNYT/videos/116561...,NaN,NaN,NaN,NaN,1.0,1.0
47850,5281959998_10150958116574999,5281959998,Two Men Entered the Ring for Their First Profe...,A reason to binge on some of our best long rea...,Anthony Taylor had won the fight. And the fall...,nytimes.com,link,shared_story,87,10,...,1.0,NaN,http://www.nytimes.com/2016/03/28/sports/boxin...,https://www.nytimes.com/2016/03/28/sports/boxi...,1.0,1.0,1.0,1.0,1.0,1.0


I can now try to join the facebook posts to articles using link, description, and name, and I can isolate only the posts that have no duplicates in each joining column.

## Exact join on link

In [188]:
len(df.loc[df['dupes_on_link'].isna()])

42001

In [192]:
# Test joining together the matches based on trimmed link, starting with only
# facebook posts that are not duplicated based on link
df_link_matches = pd.merge(left=df.loc[df['dupes_on_link'].isna()], 
                           right=df_nyt, 
                           how='inner', 
                           left_on='trim_link', 
                           right_on='web_url',
                           suffixes=('_nyt', '_nyt'))
len(df_link_matches)

36528

In [193]:
# Check if more than one NYT article matched with a single facebook link
print(len(df_link_matches[df_link_matches.duplicated(subset=['id'])]))
df_link_matches[df_link_matches.duplicated(subset=['id'], keep=False)]


13


,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,keywords,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri
1498,5281959998_350894045015656,5281959998,Q. and A. on the Papal Transition,White smoke poured from a chimney atop the Sis...,Times reporters covering the papal transition ...,thelede.blogs.nytimes.com,link,shared_story,813,102,...,[],2013-03-11T20:44:42+0000,article,,Blogs,"{'original': 'By THE NEW YORK TIMES', 'person'...",News,nyt://article/8c62f483-ffe2-5091-865c-77d2a0f9...,1915,nyt://article/8c62f483-ffe2-5091-865c-77d2a0f9...
1499,5281959998_350894045015656,5281959998,Q. and A. on the Papal Transition,White smoke poured from a chimney atop the Sis...,Times reporters covering the papal transition ...,thelede.blogs.nytimes.com,link,shared_story,813,102,...,"[{'name': 'subject', 'value': 'Cardinals (Roma...",2013-03-11T20:44:42+0000,article,,Blogs,"{'original': 'By The New York Times', 'person'...",News,nyt://article/c3d1cb31-6132-5868-81ca-911d05c2...,1915,nyt://article/c3d1cb31-6132-5868-81ca-911d05c2...
13716,5281959998_10150528737129999,5281959998,Republican Takeover of Senate Pushes Women Out...,"When Democrats lost control, women lost top jobs.",Female senators have lost high-level committee...,nyti.ms,link,shared_story,1922,375,...,"[{'name': 'subject', 'value': 'United States P...",2015-02-02T12:01:32+0000,article,National,U.S.,"{'original': 'By SHERYL GAY STOLBERG', 'person...",News,nyt://article/772a7db9-90a7-547e-a4d5-b343e19e...,1382,nyt://article/772a7db9-90a7-547e-a4d5-b343e19e...
13717,5281959998_10150528737129999,5281959998,Republican Takeover of Senate Pushes Women Out...,"When Democrats lost control, women lost top jobs.",Female senators have lost high-level committee...,nyti.ms,link,shared_story,1922,375,...,"[{'name': 'subject', 'value': 'United States P...",2015-02-02T13:26:11+0000,article,National,U.S.,"{'original': 'By Sheryl Gay Stolberg', 'person...",News,nyt://article/972cae9e-ea3c-50a1-b756-e00e7867...,1392,nyt://article/972cae9e-ea3c-50a1-b756-e00e7867...
17862,5281959998_10150599998749999,5281959998,One Year: A Century of Birthdays,The Dutch national photographer spent a year a...,Ilvy Njiokiktjien got a glimpse into Dutch cul...,lens.blogs.nytimes.com,link,shared_story,835,16,...,"[{'name': 'subject', 'value': 'Age, Chronologi...",2015-06-05T07:00:04+0000,article,,Crosswords & Games,"{'original': 'By Laurence Butet-Roch', 'person...",News,nyt://article/02d3f34e-d4a0-59b5-896b-11e56a23...,624,nyt://article/02d3f34e-d4a0-59b5-896b-11e56a23...
17863,5281959998_10150599998749999,5281959998,One Year: A Century of Birthdays,The Dutch national photographer spent a year a...,Ilvy Njiokiktjien got a glimpse into Dutch cul...,lens.blogs.nytimes.com,link,shared_story,835,16,...,[],2015-06-05T07:00:04+0000,article,,Multimedia/Photos,"{'original': 'By LAURENCE BUTET-ROCH', 'person...",News,nyt://article/ac809ee6-3ef4-5f63-8386-c1433e5a...,624,nyt://article/ac809ee6-3ef4-5f63-8386-c1433e5a...
18034,5281959998_10150602337129999,5281959998,"Live, From North Korea",'There’s probably not a better place to test t...,"During a recent trip to North Korea, the photo...",lens.blogs.nytimes.com,link,shared_story,345,11,...,[],2015-06-10T11:00:52+0000,article,,Crosswords & Games,"{'original': 'By David Gonzalez', 'person': [{...",News,nyt://article/38c4d66e-382e-5a07-9054-ea7da136...,1172,nyt://article/38c4d66e-382e-5a07-9054-ea7da136...
18035,5281959998_10150602337129999,5281959998,"Live, From North Korea",'There’s probably not a better place to test t...,"During a recent trip to North Korea, the photo...",lens.blogs.nytimes.com,link,shared_story,345,11,...,[],2015-06-10T11:00:52+0000,article,,Multimedia/Photos,"{'original': 'By DAVID GONZALEZ', 'person': [{...",News,nyt://article/77dbb645-f1bf-5da6-afa9-e2169955...,1172,nyt://article/77dbb645-f1bf-5da6-afa9-e2169955...
18224,5281959998_10150605618964999,5281959998,Timeline

In [194]:
# Check for duplicate NYT articles that were matched with Facebook posts
print(len(df_link_matches[df_link_matches.duplicated(subset=['_id'])]))
df_link_matches[df_link_matches.duplicated(subset=['_id'], keep=False)]

361


,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,keywords,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri
926,5281959998_10150275554924999,5281959998,Timeline Photos,It's official: 2012 was the hottest year ever ...,NaN,NaN,photo,added_photos,843,139,...,"[{'name': 'persons', 'value': 'Gillis, Justin'...",2013-01-08T18:33:45+0000,article,National,Science,"{'original': 'By Justin Gillis', 'person': [{'...",News,nyt://article/6b759bc5-e3df-50be-b246-3050045c...,903,nyt://article/6b759bc5-e3df-50be-b246-3050045c...
927,5281959998_473125349417587,5281959998,Not Even Close: 2012 Was Hottest Ever in U.S.,Quotation of the Day: 'The heat was remarkable...,Last year’s 55.3 degree average topped the pre...,nytimes.com,link,shared_story,212,25,...,"[{'name': 'persons', 'value': 'Gillis, Justin'...",2013-01-08T18:33:45+0000,article,National,Science,"{'original': 'By Justin Gillis', 'person': [{'...",News,nyt://article/6b759bc5-e3df-50be-b246-3050045c...,903,nyt://article/6b759bc5-e3df-50be-b246-3050045c...
938,5281959998_492250480813643,5281959998,Voters Shut Out Hall of Fame Candidates,In perhaps the most resounding referendum on t...,"For the first time since 1996, no players rece...",nytimes.com,link,shared_story,345,74,...,"[{'name': 'persons', 'value': 'Kepner, Tyler',...",2013-01-09T19:08:47+0000,article,Sports,Sports,"{'original': 'By Tyler Kepner', 'person': [{'f...",News,nyt://article/6471d8a1-5da7-51eb-96c7-e342ca06...,1345,nyt://article/6471d8a1-5da7-51eb-96c7-e342ca06...
939,5281959998_10150275861879999,5281959998,Timeline Photos,The front page of the Sports Section welcomes ...,NaN,NaN,photo,added_photos,279,23,...,"[{'name': 'persons', 'value': 'Kepner, Tyler',...",2013-01-09T19:08:47+0000,article,Sports,Sports,"{'original': 'By Tyler Kepner', 'person': [{'f...",News,nyt://article/6471d8a1-5da7-51eb-96c7-e342ca06...,1345,nyt://article/6471d8a1-5da7-51eb-96c7-e342ca06...
1585,5281959998_10150293984859999,5281959998,Timeline Photos,"“Our sons and daughters are not born to hate, ...",NaN,NaN,photo,added_photos,3694,270,...,"[{'name': 'persons', 'value': 'Obama, Barack',...",2013-03-22T09:42:45+0000,article,Foreign,World,{'original': 'By Jodi Rudoren and Mark Landler...,News,nyt://article/91c0df18-6cb0-586e-842d-35fc28da...,1360,nyt://article/91c0df18-6cb0-586e-842d-35fc28da...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36217,5281959998_10150946496639999,5281959998,Arrests and Violence at Dakota Pipeline,The tension between law enforcement and protes...,NaN,NaN,video,added_video,762,234,...,"[{'name': 'glocations', 'value': 'Standing Roc...",2016-10-28T16:37:15+0000,article,National,U.S.,{'original': 'By Sue Skalicky and Monica Davey...,News,nyt://article/cd7fec00-241a-56f9-b56e-1cdc08c2...,693,nyt://article/cd7fec00-241a-56f9-b56e-1cdc08c2...
36473,5281959998_10150955949149999,5281959998,The New York Times to Offer Open Access on Web...,We're giving everyone unlimited access to The ...,The New York Times is inviting readers to take...,nytimes.com,link,shared_story,28337,602,...,"[{'name': 'subject', 'value': 'Presidential El...",2016-11-03T20:58:39+0000,article,NewsDesk,U.S.,"{'original': '', 'person': [], 'organization':...",News,nyt://article/30fb1562-4bd8-5cda-8c44-d10f40d2...,350,nyt://article/30fb1562-4bd8-5cda-8c44-d10f40d2...
36474,5281959998_10150957229929999,5281959998,Timeline Photos,Election week is here. Enjoy unlimited access ...,NaN,NaN,photo,added_photos,1196,532,...,"[{'name': 'subject', 'value': 'Presidential El...",2016-11-03T20:58:39+0000,article,NewsDesk,U.S.,"{'original': '', 'person': [], 'organization':...",News,nyt://article/30fb1562-4bd8-5cda-8c44-d10f40d2...,350,nyt://article/30fb1562-4bd8-5cda-8c44-d10f40d2...
36491,5281959998_10150957315964999,5281959998,Where Burning Effigies (Not Just Guy Fawkes’s)...,Donald Trump and Theresa May were burned in ef...,"Bonfires, blaz

In [195]:
# update which facebook posts have been joined up, so can run further joins
# on different criteria, and only on those that aren't yet matched
df.loc[df['id'].isin(df_link_matches['id'].values), 'matched_on'] = 'link'
len(df[df['matched_on']=='link'])

36515

## Exact join on description

In [196]:
# pull only posts without duplicate descriptions, and where the posts
# haven't already been matched with an article
len(df.loc[(df['dupes_on_desc'].isna()) & (df['matched_on'].isna())])

7705

In [197]:
# Test joining together the matches based on description and snippet
df_desc_matches = pd.merge(left=df.loc[(df['dupes_on_desc'].isna()) 
                                       & (df['matched_on'].isna())], 
                           right=df_nyt, 
                           how='inner', 
                           left_on='description', 
                           right_on='snippet',
                           suffixes=('_nyt', '_nyt'))
len(df_desc_matches)

1037

In [198]:
# Check if more than one NYT article matched with a single facebook link
print(len(df_desc_matches[df_desc_matches.duplicated(subset=['id'])]))
df_desc_matches[df_desc_matches.duplicated(subset=['id'], keep=False)]


28


,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,keywords,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri
75,5281959998_10150299457829999,5281959998,Updates on Aftermath of Boston Marathon Explos...,'Something is wrong. They have been framed.' R...,The Lede is following the aftermath of Monday’...,thelede.blogs.nytimes.com,link,shared_story,342,136,...,"[{'name': 'subject', 'value': 'Boston Marathon...",2013-04-18T14:35:07+0000,article,,Blogs,"{'original': 'By Christine Hauser, Jennifer Pr...",News,nyt://article/8d63a22b-24a5-5527-a7b2-2888674d...,4192,nyt://article/8d63a22b-24a5-5527-a7b2-2888674d...
76,5281959998_10150299457829999,5281959998,Updates on Aftermath of Boston Marathon Explos...,'Something is wrong. They have been framed.' R...,The Lede is following the aftermath of Monday’...,thelede.blogs.nytimes.com,link,shared_story,342,136,...,"[{'name': 'subject', 'value': 'Boston Marathon...",2013-04-19T10:25:31+0000,article,,Blogs,"{'original': 'By The New York Times', 'person'...",News,nyt://article/c71f2857-d9ff-50c1-9cc7-3baf14e5...,13612,nyt://article/c71f2857-d9ff-50c1-9cc7-3baf14e5...
97,5281959998_10150306238154999,5281959998,Video: Bill Cunningham | Full Bloom,"Bill Cunningham, inspired by the four-foot-lon...","A focus of fashion is the length of the leg, s...",nytimes.com,link,shared_story,175,3,...,"[{'name': 'subject', 'value': 'Fashion and App...",2013-05-18T02:48:10+0000,multimedia,,Fashion & Style,"{'original': 'Joanna Nikas', 'person': [{'firs...",Video,nyt://video/854ae85f-5a35-5e2c-ba51-107b1567c087,0,nyt://video/854ae85f-5a35-5e2c-ba51-107b1567c087
98,5281959998_10150306238154999,5281959998,Video: Bill Cunningham | Full Bloom,"Bill Cunningham, inspired by the four-foot-lon...","A focus of fashion is the length of the leg, s...",nytimes.com,link,shared_story,175,3,...,"[{'name': 'subject', 'value': 'Fashion and App...",2013-05-20T16:54:55+0000,article,,Style,"{'original': 'By Bill Cunningham', 'person': [...",News,nyt://article/c4185b01-90be-5fbb-85d8-1f6f7d74...,47,nyt://article/c4185b01-90be-5fbb-85d8-1f6f7d74...
108,5281959998_10150309977124999,5281959998,Video: Escape From North Korea,'The government doesn't give us a thing but th...,A smuggler named Dragon attempts to guide Nort...,nytimes.com,link,shared_story,386,20,...,"[{'name': 'glocations', 'value': 'North Korea'...",2013-06-06T01:05:08+0000,multimedia,Opinion,Opinion,"{'original': 'Ann Shin', 'person': [{'firstnam...",Video,nyt://video/786833f5-cd66-5444-9cdf-75707417146a,0,nyt://video/786833f5-cd66-5444-9cdf-75707417146a
109,5281959998_10150309977124999,5281959998,Video: Escape From North Korea,'The government doesn't give us a thing but th...,A smuggler named Dragon attempts to guide Nort...,nytimes.com,link,shared_story,386,20,...,"[{'name': 'glocations', 'value': 'North Korea'...",2013-06-06T01:12:22+0000,article,OpEd,Opinion,"{'original': 'By Ann Shin', 'person': [{'first...",Op-Ed,nyt://article/779326d9-6752-5ff6-95a3-31ef22ba...,201,nyt://article/779326d9-6752-5ff6-95a3-31ef22ba...
137,5281959998_10150324158779999,5281959998,Video: Tunnel Vision,Was your commute this morning this transporting?,A short impressionistic film shot from New Yor...,nytimes.com,link,shared_story,186,12,...,"[{'name': 'subject', 'value': 'Subways', 'rank...",2013-07-30T01:10:14+0000,multimedia,Opinion,Opinion,"{'original': 'Jeff Scher', 'person': [{'firstn...",Video,nyt://video/2222d988-9f30-5621-99fe-11d5c7df296b,0,nyt://video/2222d988-9f30-5621-99fe-11d5c7df296b
138,5281959998_10150324158779999,5281959998,Video: Tunnel Vision,Was your commute this morning this transporting?,A short impressionistic film shot from New Yor...,nytimes.com,link,shared_story,186,12,...,[],2013-07-30T01:12:34+0000,article,,Opinion,"{'original': 'By Jeff Scher', 'person': [{'fir...",News,nyt://article/ac28e61c-7368-55fa-93ee-994b60de...,340,nyt://article/ac28e61c-7368-55fa-93ee-994b60de...
188,

It looks like these duplicates are primarily due to there being a video version and a news/written version of certain pieces.

Looking at a few examples, can I easily tell whether the facebook post is for one or the other? I may need to write my own custom deduplication script to identify the best one to keep, especially if only certain ones will have comments I can use to compare to facebook likes and shares.

In [199]:
# Check for duplicate NYT articles that were matched with Facebook posts
print(len(df_desc_matches[df_desc_matches.duplicated(subset=['_id'])]))
df_desc_matches[df_desc_matches.duplicated(subset=['_id'], keep=False)]

0


,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,keywords,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri


In [200]:
# update which facebook posts have been joined up, so can run further joins
# on different criteria, and only on those that aren't yet matched
df.loc[df['id'].isin(df_desc_matches['id'].values), 'matched_on'] = 'desc'
len(df[df['matched_on']=='desc'])

1009

In [201]:
# how many unmatched are left?
len(df[df['matched_on'].isna()])

10343

In [202]:
# how many unmatched are not duplicates on something?
len(df[(df['matched_on'].isna()) & (
    ((df['dupes_on_link'].isna()) & 
    (df['dupes_on_desc'].isna()) & 
    (df['dupes_on_name'].isna()))
)])


2737

In [203]:
df[(df['matched_on'].isna()) & (
    ((df['dupes_on_link'].isna()) & 
    (df['dupes_on_desc'].isna()) & 
    (df['dupes_on_name'].isna()))
)]

,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,link_expanded,link_backup,trim_link,dupes_on_link,dupe_link_child,dupes_on_desc,dupe_desc_child,dupes_on_name,dupe_name_child,matched_on
2,5281959998_467433813289503,5281959998,The Organic Fable,"'Organic has long since become an ideology, th...",A study exposes the hype behind a pseudoscient...,nytimes.com,link,shared_story,812,290,...,NaN,http://www.nytimes.com/2012/09/07/opinion/roge...,https://www.nytimes.com/2012/09/07/opinion/rog...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,5281959998_269424823169279,5281959998,Jessica Chastain TimesTalks Live Webcast,NaN,Watch the free livestream here: http://nyti.ms...,Jessica Chastain TimesTalks Live Webcast,event,created_event,51,7,...,NaN,https://www.facebook.com/events/269424823169279/,https://www.facebook.com/events/269424823169279/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,5281959998_380912158643622,5281959998,'The Schools for Tomorrow' Live Webcast,NaN,Watch the free livestream of our second annual...,'The Schools for Tomorrow' Live Webcast,event,created_event,49,7,...,NaN,https://www.facebook.com/events/380912158643622/,https://www.facebook.com/events/380912158643622/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,5281959998_224679674326401,5281959998,Video: Patriot Game,'POWER YOUR PRESIDENT TO WINNING. USE MANY GOO...,The Gregory Brothers present a video game-insp...,nytimes.com,link,shared_story,178,38,...,NaN,http://www.nytimes.com/video/2012/09/16/opinio...,https://www.nytimes.com/video/2012/09/16/opini...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158,5281959998_10150255185794999,5281959998,NaN,We are currently working with Facebook to reso...,NaN,NaN,status,mobile_status_update,495,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47844,5281959998_10150957984614999,5281959998,Instagram photo by New York Times Archives • N...,"1917: “Women of all ages, from the nearly feeb...",See this Instagram photo by @nytarchives • 63 ...,instagram.com,link,shared_story,23320,257,...,NaN,https://www.instagram.com/p/BMgr8zwBZwQ/,https://www.instagram.com/p/BMgr8zwBZwQ/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47848,5281959998_10150958024099999,5281959998,Whole-Roasted Stuffed Squash Recipe,A vegetarian main to rival every other dish at...,Here is a vegetarian dinner course of impressi...,cooking.nytimes.com,link,shared_story,277,52,...,NaN,http://cooking.nytimes.com/recipes/1017780-who...,https://cooking.nytimes.com/recipes/1017780-wh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47853,5281959998_10150958141049999,5281959998,Naturalization: Becoming an American,Carmen Joan Edwards spent 15 years working tow...,NaN,NaN,video,added_video,3764,147,...,NaN,https://www.facebook.com/nytimes/videos/101509...,https://www.facebook.com/nytimes/videos/101509...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47859,5281959998_10150958344969999,5281959998,2016 Election Puppet Show,We're putting on a live puppet show re-enactin...,NaN,NaN,video,added_video,1211,802,...,NaN,https://www.facebook.com/nytimes/videos/101509...,https://www.facebook.com/nytimes/videos/101509...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Exact join on headline

In [204]:
# separate out main headline from headline dict column
df_nyt['main_headline'] = df_nyt['headline'].map(lambda x: x['main'])

In [205]:
# pull only posts without duplicate names, and where the posts
# haven't already been matched with an article
len(df.loc[(df['dupes_on_name'].isna()) & (df['matched_on'].isna())])

5084

In [206]:
# Test joining together the matches based on description and snippet
df_name_matches = pd.merge(left=df.loc[(df['dupes_on_name'].isna()) 
                                       & (df['matched_on'].isna())], 
                           right=df_nyt, 
                           how='inner', 
                           left_on='name', 
                           right_on='main_headline',
                           suffixes=('_nyt', '_nyt'))
len(df_name_matches)

702

In [207]:
# Check if more than one NYT article matched with a single facebook link
print(len(df_name_matches[df_name_matches.duplicated(subset=['id'])]))
df_name_matches[df_name_matches.duplicated(subset=['id'], keep=False)]


159


,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,main_headline
1,5281959998_423902177671528,5281959998,Global Citizen Festival,"Beginning at 4:00pm ET, watch the free live we...","On Sept. 29 at 4:30 p.m. EST, watch a live vid...",nytimes.com,link,shared_story,177,10,...,2012-09-30T23:21:31+0000,multimedia,Arts,Arts,"{'original': '', 'person': [], 'organization':...",Slideshow,nyt://slideshow/a923a469-ffbb-56a1-bfa8-d69b10...,0,nyt://slideshow/a923a469-ffbb-56a1-bfa8-d69b10...,Global Citizen Festival
2,5281959998_423902177671528,5281959998,Global Citizen Festival,"Beginning at 4:00pm ET, watch the free live we...","On Sept. 29 at 4:30 p.m. EST, watch a live vid...",nytimes.com,link,shared_story,177,10,...,2013-09-29T21:57:57+0000,multimedia,Arts,Arts,"{'original': '', 'person': [], 'organization':...",Slideshow,nyt://slideshow/2f1ad1cc-da0f-596b-b7fc-90e594...,0,nyt://slideshow/2f1ad1cc-da0f-596b-b7fc-90e594...,Global Citizen Festival
3,5281959998_423902177671528,5281959998,Global Citizen Festival,"Beginning at 4:00pm ET, watch the free live we...","On Sept. 29 at 4:30 p.m. EST, watch a live vid...",nytimes.com,link,shared_story,177,10,...,2015-09-27T21:11:14+0000,multimedia,Arts,Arts,"{'original': '', 'person': [], 'organization':...",Slideshow,nyt://slideshow/abc08ebc-c02a-5aac-8780-d4698c...,0,nyt://slideshow/abc08ebc-c02a-5aac-8780-d4698c...,Global Citizen Festival
58,5281959998_10150349642114999,5281959998,Forging an Art Market in China,"As China’s booming art market has grown, so ha...","In ChinaÆs growing art market, now the second ...",nytimes.com,link,shared_story,591,31,...,2013-10-28T01:29:52+0000,article,Culture,Arts,"{'original': 'By David Barboza, Graham Bowley ...",News,nyt://article/527eacca-dbe4-5665-9a6f-8b383cf8...,4613,nyt://article/527eacca-dbe4-5665-9a6f-8b383cf8...,Forging an Art Market in China
59,5281959998_10150349642114999,5281959998,Forging an Art Market in China,"As China’s booming art market has grown, so ha...","In ChinaÆs growing art market, now the second ...",nytimes.com,link,shared_story,591,31,...,2014-02-18T16:28:40+0000,multimedia,World,World,"{'original': '', 'person': [], 'organization':...",Interactive Feature,nyt://interactive/f366afe2-49ea-521d-bf15-32b9...,0,nyt://interactive/f366afe2-49ea-521d-bf15-32b9...,Forging an Art Market in China
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,5281959998_10150854190719999,5281959998,Deadly Shooting at Munich Mall,"A gunman opened fire at a Munich mall, killing...",NaN,NaN,video,added_video,640,213,...,2016-07-23T11:56:16+0000,multimedia,World,World,"{'original': '', 'person': [], 'organization':...",Slideshow,nyt://slideshow/6a53571b-847f-56ca-a6bc-b24c2e...,0,nyt://slideshow/6a53571b-847f-56ca-a6bc-b24c2e...,Deadly Shooting at Munich Mall
640,5281959998_10150856491074999,5281959998,The Trans-Pacific Partnership Trade Accord Exp...,Need a rundown on the Trans-Pacific Partnershi...,"The giant trade pact, years in the making, has...",nytimes.com,link,shared_story,256,52,...,2015-10-05T15:26:07+0000,article,Business,Business Day,"{'original': 'By Kevin Granville', 'person': [...",News,nyt://article/f2fe8d24-7542-5e40-b7c1-aceff307...,1327,nyt://article/f2fe8d24-7542-5e40-b7c1-aceff307...,The Trans-Pacific Partnership Trade Accord Exp...
641,5281959998_10150856491074999,5281959998,The Trans-Pacific Partnership Trade Accord Exp...,Need a rundown on the Trans-Pacific Partnershi...,"The giant trade pact, years in the making, has...",nytimes.com,link,shared_story,256,52,...,2016-07-26T22:36:25+0000,article,Business,Business Day,"{'original': 'By Kevin Granville', 'person': [...",News,nyt://article/7f176cc0-5c9f-5d11-98a1-1adfdbe3...,1326,nyt://article/7f176cc0-5c9f-5d11-98a1-1adfdbe3...,The Trans-Pacific Partnership Trade Accord Exp...
652,5281959998_10150859876799999,528

These look like duplicates for the same reasons the others were; multiple copies of a feature as different types of media, or perhaps updated like with slideshows where there may have been multiple versions.

In [208]:
# Check for duplicate NYT articles that were matched with Facebook posts
print(len(df_name_matches[df_name_matches.duplicated(subset=['_id'])]))
df_name_matches[df_name_matches.duplicated(subset=['_id'], keep=False)]

0


,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,main_headline


In [209]:
# update which facebook posts have been joined up, so can run further joins
# on different criteria, and only on those that aren't yet matched
df.loc[df['id'].isin(df_name_matches['id'].values), 'matched_on'] = 'name'
len(df[df['matched_on']=='name'])

543

In [210]:
# how many unmatched are left?
len(df[df['matched_on'].isna()])

9800

In [211]:
# unmatched and not duplicates of any sort I've tested
df[(df['matched_on'].isna()) & (
    ((df['dupes_on_link'].isna()) & 
    (df['dupes_on_desc'].isna()) & 
    (df['dupes_on_name'].isna()))
)]

,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,link_expanded,link_backup,trim_link,dupes_on_link,dupe_link_child,dupes_on_desc,dupe_desc_child,dupes_on_name,dupe_name_child,matched_on
2,5281959998_467433813289503,5281959998,The Organic Fable,"'Organic has long since become an ideology, th...",A study exposes the hype behind a pseudoscient...,nytimes.com,link,shared_story,812,290,...,NaN,http://www.nytimes.com/2012/09/07/opinion/roge...,https://www.nytimes.com/2012/09/07/opinion/rog...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,5281959998_269424823169279,5281959998,Jessica Chastain TimesTalks Live Webcast,NaN,Watch the free livestream here: http://nyti.ms...,Jessica Chastain TimesTalks Live Webcast,event,created_event,51,7,...,NaN,https://www.facebook.com/events/269424823169279/,https://www.facebook.com/events/269424823169279/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,5281959998_380912158643622,5281959998,'The Schools for Tomorrow' Live Webcast,NaN,Watch the free livestream of our second annual...,'The Schools for Tomorrow' Live Webcast,event,created_event,49,7,...,NaN,https://www.facebook.com/events/380912158643622/,https://www.facebook.com/events/380912158643622/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,5281959998_224679674326401,5281959998,Video: Patriot Game,'POWER YOUR PRESIDENT TO WINNING. USE MANY GOO...,The Gregory Brothers present a video game-insp...,nytimes.com,link,shared_story,178,38,...,NaN,http://www.nytimes.com/video/2012/09/16/opinio...,https://www.nytimes.com/video/2012/09/16/opini...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158,5281959998_10150255185794999,5281959998,NaN,We are currently working with Facebook to reso...,NaN,NaN,status,mobile_status_update,495,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47836,5281959998_10150957195084999,5281959998,Ride Along With 'Souls to the Polls',"360° video: In North Carolina, where the presi...",NaN,NaN,video,added_video,995,83,...,NaN,https://www.facebook.com/nytimes/videos/101509...,https://www.facebook.com/nytimes/videos/101509...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47844,5281959998_10150957984614999,5281959998,Instagram photo by New York Times Archives • N...,"1917: “Women of all ages, from the nearly feeb...",See this Instagram photo by @nytarchives • 63 ...,instagram.com,link,shared_story,23320,257,...,NaN,https://www.instagram.com/p/BMgr8zwBZwQ/,https://www.instagram.com/p/BMgr8zwBZwQ/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47848,5281959998_10150958024099999,5281959998,Whole-Roasted Stuffed Squash Recipe,A vegetarian main to rival every other dish at...,Here is a vegetarian dinner course of impressi...,cooking.nytimes.com,link,shared_story,277,52,...,NaN,http://cooking.nytimes.com/recipes/1017780-who...,https://cooking.nytimes.com/recipes/1017780-wh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47859,5281959998_10150958344969999,5281959998,2016 Election Puppet Show,We're putting on a live puppet show re-enactin...,NaN,NaN,video,added_video,1211,802,...,NaN,https://www.facebook.com/nytimes/videos/101509...,https://www.facebook.com/nytimes/videos/101509...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Pull dates from link

As a final step, I'm going to try to pull the year-month-day out of the links in both the Facebook and NYT API links, and match where both the description/name and the date from the link matches.

The publication date and date when an article was posted to Facebook sometimes varies, since the articles are sometimes posted to Facebook the next day or later, so I think using the date in the link should be the best match.

### Facebook posts

In [212]:
df = nlp_prep.pattern_match_in_df(df, doc_col='trim_link', hit_col='link_date', 
                pattern=r"/(201[2-6]/[01][0-9]/[0-3][0-9])/", 
                out_type='string', replace=False)

In [213]:
df[['trim_link', 'link_date']].tail()

,trim_link,link_date
47862,https://www.nytimes.com/2016/11/06/arts/televi...,2016/11/06
47863,https://www.nytimes.com/2016/11/07/opinion/cam...,2016/11/07
47864,https://www.nytimes.com/2016/11/08/upshot/this...,2016/11/08
47865,https://www.nytimes.com/watching/recommendatio...,
47866,https://www.nytimes.com/2016/11/07/opinion/cam...,2016/11/07


### NYT articles

In [214]:
df_nyt = nlp_prep.pattern_match_in_df(df_nyt, doc_col='web_url', 
                hit_col='link_date', 
                pattern=r"/(201[2-6]/[01][0-9]/[0-3][0-9])/", 
                out_type='string', replace=False)

In [216]:
df_nyt[['web_url', 'link_date']].tail()

,web_url,link_date
425926,https://www.nytimes.com/2016/12/31/world/middl...,2016/12/31
425927,https://www.nytimes.com/2016/12/31/business/f-...,2016/12/31
425928,https://www.nytimes.com/2016/12/31/crosswords/...,2016/12/31
425929,https://www.nytimes.com/2016/12/31/us/politics...,2016/12/31
425930,https://www.nytimes.com/2016/12/31/world/europ...,2016/12/31


## Join on link dates and other fields

In [217]:
df_name_matches2 = pd.merge(left=df.loc[(df['matched_on'].isna()) &
                                        (df['link_date'].isna()==False) &
                                        (df['link_date'] != '')], 
                           right=df_nyt, 
                           how='inner', 
                           left_on=['name', 'link_date'], 
                           right_on=['main_headline', 'link_date'],
                           suffixes=('_nyt', '_nyt'))
len(df_name_matches2)

1739

In [218]:
df_name_matches2[['message', 'description', 'name', 'snippet', 'likes_count',
                  'comments_count', 'shares_count',
                  'main_headline', 'link_date', 'posted_at']]

,message,description,name,snippet,likes_count,comments_count,shares_count,main_headline,link_date,posted_at
0,"When it's time for a friendship to end, is it ...","How much better things were 50 years ago, when...","Unfriending Someone, Before Facebook","How much better things were 50 years ago, when...",454,41,195,"Unfriending Someone, Before Facebook",2012/10/03,2012-10-03 16:10:40
1,Is unfriending a person on Facebook the right ...,"How much better things were 50 years ago, when...","Unfriending Someone, Before Facebook","How much better things were 50 years ago, when...",461,69,196,"Unfriending Someone, Before Facebook",2012/10/03,2012-10-04 20:17:42
2,Select President Barack Obama or Mitt Romney i...,Explore the routes through the electoral battl...,512 Paths to the White House,Explore the routes through the electoral battl...,3605,578,1435,512 Paths to the White House,2012/11/02,2012-11-03 17:36:30
3,Obama and Romney have 512 paths to the White H...,Explore the routes through the electoral battl...,512 Paths to the White House,Explore the routes through the electoral battl...,3783,506,1177,512 Paths to the White House,2012/11/02,2012-11-06 16:49:58
4,Opponents of President Mohamed Morsi were repo...,Protesters were said to have set fire to the o...,Clashes Break Out After Morsi Seizes New Power...,Thousands of protesters chanted for the downfa...,194,22,40,Clashes Break Out After Morsi Seizes New Power...,2012/11/24,2012-11-23 16:11:39
...,...,...,...,...,...,...,...,...,...,...
1734,"'Donald J. Trump has done one good thing,' Joh...",John Leguizamo guest edits this week’s Race/Re...,Making Latino Life Visible,John Leguizamo guest edits this week’s Race/Re...,403,51,53,Making Latino Life Visible,2016/10/21,2016-10-21 19:25:00
1735,'Latinos need to demand our place in American ...,John Leguizamo guest edits this week’s Race/Re...,Making Latino Life Visible,John Leguizamo guest edits this week’s Race/Re...,477,46,102,Making Latino Life Visible,2016/10/21,2016-10-24 02:15:00
1736,Estonia is using a form of 'military sport' to...,'The guerrilla activity should start on occupi...,"Spooked by Russia, Tiny Estonia Trains a Natio...","A NATO member, Estonia is using an improbably ...",1551,124,675,"Spooked by Russia, Tiny Estonia Trains a Natio...",2016/11/01,2016-11-01 07:25:00
1737,Estonia is using a form of 'military sport' to...,“The guerrilla activity should start on occupi...,"Spooked by Russia, Tiny Estonia Trains a Natio...","A NATO member, Estonia is using an improbably ...",681,55,256,"Spooked by Russia, Tiny Estonia Trains a Natio...",2016/11/01,2016-11-01 08:25:01


In [219]:
# update matched status for new name matches

df.loc[df['id'].isin(df_name_matches2['id'].values), 'matched_on'] = 'name'

In [220]:
df_desc_matches2 = pd.merge(left=df.loc[(df['matched_on'].isna()) &
                                        (df['link_date'].isna()==False) &
                                        (df['link_date'] != '')], 
                           right=df_nyt, 
                           how='inner', 
                           left_on=['description', 'link_date'], 
                           right_on=['snippet', 'link_date'],
                           suffixes=('_nyt', '_nyt'))
len(df_desc_matches2)

759

In [221]:
df_desc_matches2[['message', 'description', 'name', 'snippet', 'likes_count',
                  'comments_count', 'shares_count',
                  'main_headline', 'link_date', 'posted_at']]

,message,description,name,snippet,likes_count,comments_count,shares_count,main_headline,link_date,posted_at
0,The U.S. economy grew at an annual rate of 2 p...,More positive consumer activity and a healthie...,U.S. Economy Grew at 2% Rate in 3rd Quarter,More positive consumer activity and a healthie...,730,72,165,U.S. Growth Rate Picks Up to 2%,2012/10/27,2012-10-26 12:53:54
1,Does a puritanical American culture overreact ...,The F.B.I. investigation that toppled David H....,Petraeus Case Raises Fears About Privacy in Di...,The F.B.I. investigation that toppled David H....,600,175,133,Online Privacy Issue Is Also in Play in Petrae...,2012/11/14,2012-11-13 22:20:02
2,How should you respond if you are at the scene...,People at the scene of an attack are more like...,Police Now Advise Assertive Response to Mass A...,People at the scene of an attack are more like...,451,136,170,"In Shift, Police Advise Taking an Active Role ...",2013/04/07,2013-04-06 23:48:34
3,On Sunday The Guardian revealed that Edward Sn...,A British newspaper identified its source as E...,Former C.I.A. Worker Says He Leaked Surveillan...,A British newspaper identified its source as E...,691,122,132,Ex-Worker at C.I.A. Says He Leaked Data on Sur...,2013/06/10,2013-06-09 19:14:41
4,Federal health officials said Wednesday that t...,The prevalence of dangerous strains of the hum...,Study Finds Sharp Drop in HPV Infections in Girls,The prevalence of dangerous strains of the hum...,975,51,257,HPV Vaccine Is Credited in Fall of Teenagers’ ...,2013/06/20,2013-06-19 17:39:19
...,...,...,...,...,...,...,...,...,...,...
754,“Cold water surfing is the new black.”,There’s a thriving surf scene in Norway’s Lofo...,"6,000 Miles North of Hawaii",There’s a thriving surf scene in Norway’s Lofo...,249,33,46,Surfing Under the Northern Lights,2016/10/23,2016-10-18 16:25:32
755,"'The first time I was here, my heart was pound...",There’s a thriving surf scene in Norway’s Lofo...,Surfing 100 Miles Above the Arctic Circle,There’s a thriving surf scene in Norway’s Lofo...,282,20,66,Surfing Under the Northern Lights,2016/10/23,2016-10-24 01:23:16
756,"Use them properly, and these tricks will almos...",These are outlier tips — but when used properl...,3 airfare hacks for the committed penny-pincher,These are outlier tips — but when used properl...,337,38,112,3 Airfare Hacks for the Committed Penny-Pincher,2016/05/29,2016-10-18 21:40:00
757,'The culture of a workplace … has a huge impac...,Corporate culture is more important to your ha...,The one question you should ask about every ne...,Corporate culture is more important to your ha...,3684,158,1534,The One Question You Should Ask About Every Ne...,2015/12/20,2016-10-19 02:40:00


In [222]:
# update matched status for new desc matches

df.loc[df['id'].isin(df_desc_matches2['id'].values), 'matched_on'] = 'desc'

In [223]:
# get matches even on duplicate links. Duplicate links should definitely be the
# same article
df_link_matches2 = pd.merge(left=df.loc[(df['matched_on'].isna())], 
                           right=df_nyt, 
                           how='inner', 
                           left_on='trim_link', 
                           right_on='web_url',
                           suffixes=('_nyt', '_nyt'))
len(df_link_matches2)

1664

In [224]:
# update matched status for new link matches
df.loc[df['id'].isin(df_link_matches2['id'].values), 'matched_on'] = 'link'

## Looking at Dupes Again

In [225]:
# how many unmatched are left?
len(df[df['matched_on'].isna()])

5658

In [226]:
# how many are unmatched and not dupes are left?
len(df.loc[(df['matched_on'].isna()) & (
    ((df['dupes_on_link'].isna()) & 
    (df['dupes_on_desc'].isna()) & 
    (df['dupes_on_name'].isna())))])

2580

In [227]:
# check out unmatched that are some type of dupe
df.loc[(df['matched_on'].isna()) & 
    (df['dupes_on_link'].isna()==False) & 
    (df['dupes_on_desc'].isna()==False) & 
    (df['dupes_on_name'].isna()==False) & (df['caption']=='nytimes.com'), 
      ['message', 'description', 'name', 'posted_at', 'dupes_on_desc', 
       'dupes_on_name', 'trim_link']]

,message,description,name,posted_at,dupes_on_desc,dupes_on_name,trim_link
1037,Special ReportA group of world-class skiers an...,Fresh powder beckoned 16 expert skiers and sno...,Snow Fall: The Avalanche at Tunnel Creek,2012-12-20 21:40:06,1.0,1.0,https://www.nytimes.com/projects/2012/snow-fall/
1054,"John Branch, author of 'Snow Fall: The Avalanc...",Fresh powder beckoned 16 expert skiers and sno...,Snow Fall: The Avalanche at Tunnel Creek,2012-12-22 18:14:04,1.0,1.0,https://www.nytimes.com/projects/2012/snow-fall/
4393,Readers around the world shared their photogra...,Photographs and personal reflections from peop...,Your Stories of Life in High-Rises,2013-10-18 20:41:48,1.0,1.0,https://www.nytimes.com/projects/2013/high-ris...
4508,Dig into a new batch of photos and reflections...,Photographs and personal reflections from peop...,Your Stories of Life in High-Rises,2013-10-29 22:52:51,1.0,1.0,https://www.nytimes.com/projects/2013/high-ris...
5239,We asked for your photos and stories about lov...,The New York Times Magazine invited readers to...,The Lives They Loved,2013-12-26 17:30:29,1.0,1.0,https://www.nytimes.com/projects/2013/lives-th...
5298,We invited you to contribute a photo and a sto...,The New York Times Magazine invited readers to...,The Lives They Loved,2013-12-31 20:30:00,1.0,1.0,https://www.nytimes.com/projects/2013/lives-th...
6267,Follow along for all things Oscar.,Join us Sunday for all things Oscar: beginning...,Live Coverage of the 86th Academy Awards,2014-03-03 01:31:57,1.0,1.0,https://www.nytimes.com/projects/oscars/2014/i...
6268,Is Jennifer Lawrence better at walking down re...,Join us Sunday for all things Oscar: beginning...,Live Coverage of the 86th Academy Awards,2014-03-03 01:38:47,1.0,1.0,https://www.nytimes.com/projects/oscars/2014/i...
13678,"The chief executive of the oil giant Total, Ch...","Christophe de Margerie, who became C.E.O. in 2...",Total Oil Chief Dies When Jet Hits Plow,2014-10-21 03:23:33,1.0,1.0,https://www.nytimes.com/2014/10/21/business/in...
15417,"Thanksgiving 101: Turkey how-to's, time-saving...",Answers to common Thanksgiving cooking questions.,How to Cook Thanksgiving Dinner: Frequently As...,2014-11-21 22:55:00,1.0,1.0,https://cooking.nytimes.com/thanksgiving/dinne...


In [228]:
df_name_matches3 = pd.merge(left=df.loc[(df['matched_on'].isna())], 
                           right=df_nyt, 
                           how='inner', 
                           left_on='name', 
                           right_on='main_headline',
                           suffixes=('_nyt', '_nyt'))
len(df_name_matches3)

372

In [229]:
# Review duplicates where we have the same message
# this means the facebook entries are duplicated, so we found more than one
# match on name
df_name_matches3[df_name_matches3.duplicated(subset=['id'], keep=False)]

,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,main_headline,link_date_nyt
3,5281959998_398719286868912,5281959998,The Lives They Loved,We asked for your photos and stories about lov...,Readers’ photographs of people close to them w...,nytimes.com,link,shared_story,241,26,...,multimedia,Magazine,Magazine,"{'original': 'By THE NEW YORK TIMES', 'person'...",Interactive Feature,nyt://interactive/04076814-354c-5a5c-94d6-8579...,0,nyt://interactive/04076814-354c-5a5c-94d6-8579...,The Lives They Loved,
4,5281959998_398719286868912,5281959998,The Lives They Loved,We asked for your photos and stories about lov...,Readers’ photographs of people close to them w...,nytimes.com,link,shared_story,241,26,...,multimedia,Magazine,Magazine,"{'original': 'By THE NEW YORK TIMES', 'person'...",Interactive Feature,nyt://interactive/0fde51ee-8a41-5806-a2b0-0113...,0,nyt://interactive/0fde51ee-8a41-5806-a2b0-0113...,The Lives They Loved,2016/12/22
5,5281959998_10150369761694999,5281959998,The Lives They Loved,We asked for your photos and stories about lov...,The New York Times Magazine invited readers to...,nytimes.com,link,shared_story,1102,41,...,multimedia,Magazine,Magazine,"{'original': 'By THE NEW YORK TIMES', 'person'...",Interactive Feature,nyt://interactive/04076814-354c-5a5c-94d6-8579...,0,nyt://interactive/04076814-354c-5a5c-94d6-8579...,The Lives They Loved,
6,5281959998_10150369761694999,5281959998,The Lives They Loved,We asked for your photos and stories about lov...,The New York Times Magazine invited readers to...,nytimes.com,link,shared_story,1102,41,...,multimedia,Magazine,Magazine,"{'original': 'By THE NEW YORK TIMES', 'person'...",Interactive Feature,nyt://interactive/0fde51ee-8a41-5806-a2b0-0113...,0,nyt://interactive/0fde51ee-8a41-5806-a2b0-0113...,The Lives They Loved,2016/12/22
7,5281959998_10150371246169999,5281959998,The Lives They Loved,We invited you to contribute a photo and a sto...,The New York Times Magazine invited readers to...,nytimes.com,link,shared_story,929,61,...,multimedia,Magazine,Magazine,"{'original': 'By THE NEW YORK TIMES', 'person'...",Interactive Feature,nyt://interactive/04076814-354c-5a5c-94d6-8579...,0,nyt://interactive/04076814-354c-5a5c-94d6-8579...,The Lives They Loved,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,5281959998_10150760436579999,5281959998,Unpublished Black History,Do you know these moments in sports history?,NaN,NaN,video,added_video,546,38,...,multimedia,The Upshot,The Upshot,"{'original': '', 'person': [], 'organization':...",Slideshow,nyt://slideshow/82633b8e-bfdf-5fc9-a909-77e9a4...,0,nyt://slideshow/82633b8e-bfdf-5fc9-a909-77e9a4...,Unpublished Black History,2016/01/31
364,5281959998_10150771498819999,5281959998,Unpublished Black History,Readers share vintage photos of Tuskegee Airme...,After we unearthed dozens of photographs from ...,nytimes.com,link,shared_story,507,16,...,article,National,U.S.,"{'original': 'By The New York Times', 'person'...",News,nyt://article/178c109b-562d-57e2-b80b-3bdd597d...,71,nyt://article/178c109b-562d-57e2-b80b-3bdd597d...,Unpublished Black History,2016/01/31
365,5281959998_10150771498819999,5281959998,Unpublished Black History,Readers share vintage photos of Tuskegee Airme...,After we unearthed dozens of photographs from ...,nytimes.com,link,shared_story,507,16,...,multimedia,The Upshot,The Upshot,"{'original': '', 'person': [], 'organization':...",Slideshow,nyt://slideshow/82633b8e-bfdf-5fc9-a909-77e9a4...,0,nyt://slideshow/82633b8e-bfdf-5fc9-a909-77e9a4...,Unpublished Black History,2016/01/31
369,5281959998_10150925069114999,5281959998,The Presidential Debate in Song: Who’s Gonna W...,Joseph Gordon-Levitt and the The Gregory Broth...,NaN,NaN,video,added_video,6233,643,...,multimedia,,Opinion,{'original': 'By THE GREGORY BROTHERS and JOSE...,Video,nyt://video/010760ee-69

In [230]:
df_name_matches3[['message', 'description', 'name', 'snippet', 'likes_count',
                  'comments_count', 'shares_count',
                  'main_headline', 'posted_at', 'pub_date']]

,message,description,name,snippet,likes_count,comments_count,shares_count,main_headline,posted_at,pub_date
0,"On Wednesday, here in our Facebook feed, we pu...",The first passage of New York Times reporter J...,Snow Fall: The Avalanche at Tunnel Creek,A group of world-class skiers and snowboarders...,231,14,109,Snow Fall: The Avalanche at Tunnel Creek,2012-12-20 05:38:48,2012-12-20T20:48:33+0000
1,Special ReportA group of world-class skiers an...,Fresh powder beckoned 16 expert skiers and sno...,Snow Fall: The Avalanche at Tunnel Creek,A group of world-class skiers and snowboarders...,538,64,322,Snow Fall: The Avalanche at Tunnel Creek,2012-12-20 21:40:06,2012-12-20T20:48:33+0000
2,"John Branch, author of 'Snow Fall: The Avalanc...",Fresh powder beckoned 16 expert skiers and sno...,Snow Fall: The Avalanche at Tunnel Creek,A group of world-class skiers and snowboarders...,175,11,30,Snow Fall: The Avalanche at Tunnel Creek,2012-12-22 18:14:04,2012-12-20T20:48:33+0000
3,We asked for your photos and stories about lov...,Readers’ photographs of people close to them w...,The Lives They Loved,As part of the magazine’s annual The Lives The...,241,26,71,The Lives They Loved,2012-12-27 23:42:19,2015-12-23T04:21:28+0000
4,We asked for your photos and stories about lov...,Readers’ photographs of people close to them w...,The Lives They Loved,As part of the magazine’s annual The Lives The...,241,26,71,The Lives They Loved,2012-12-27 23:42:19,2016-12-22T20:18:17+0000
...,...,...,...,...,...,...,...,...,...,...
367,Best of luck to runners participating in today...,Starting a new running habit doesn’t have to b...,How to Start Running,"Running is a great way to get fit, feel better...",85,19,33,How to Start Running,2016-04-18 13:41:45,2016-08-23T20:38:59+0000
368,Shigeaki Mori was 8 years old when the United ...,NaN,"Hiroshima Survivor Cries, and Obama Gives Him ...","Shigeaki Mori, who was 8 when the United State...",14605,560,3750,"Hiroshima Survivor Cries, and Obama Gives Him ...",2016-05-27 18:25:00,2016-05-27T16:30:01+0000
369,Joseph Gordon-Levitt and the The Gregory Broth...,NaN,The Presidential Debate in Song: Who’s Gonna W...,Joseph Gordon-Levitt and the Gregory Brothers ...,6233,643,5836,The Presidential Debate in Song: Who’s Gonna W...,2016-10-11 09:25:00,2016-10-10T22:50:36+0000
370,Joseph Gordon-Levitt and the The Gregory Broth...,NaN,The Presidential Debate in Song: Who’s Gonna W...,Joseph Gordon-Levitt and the Gregory Brothers ...,6233,643,5836,The Presidential Debate in Song: Who’s Gonna W...,2016-10-11 09:25:00,2016-10-10T22:53:12+0000


In [231]:
df_name_matches3['name'].value_counts()[:30]

What to Cook This Week                                                          295
Pictures of the Day: Hong Kong and Elsewhere                                     16
The Lives They Loved                                                              8
Unpublished Black History                                                         8
Netanyahu Addresses Congress on Iran                                              5
Pictures of the Day: Australia and Elsewhere                                      4
The Lives They Lived                                                              4
Snow Fall: The Avalanche at Tunnel Creek                                          3
Transgender Today                                                                 3
How to Cook Salmon                                                                3
Inside Fashion Week                                                               2
Faces of Breast Cancer                                                      

## Pull posted/published dates

Before I expanded the links, the majority of duplicates based on name were recurring features, not instances where the same link was posted twice. I didn't see anything unique about the message, name, or description that I could use to match, and the links often weren't matching even when they aren't shortened.

After expanding the links, I had a lot more entries match based on link, especially since I could pull the date out of the link itself.

However, as a last effort, I'm going to pull the date posted/published and use that to match as an alternative to the date in the link itself.

### Facebook posts

In [232]:
df = nlp_prep.pattern_match_in_df(df, doc_col='posted_at', 
                                  hit_col='post_date', 
                pattern=r"(201[2-6]-[01][0-9]-[0-3][0-9])", 
                out_type='string', replace=False)

### NYT Articles

In [233]:
df_nyt = nlp_prep.pattern_match_in_df(df_nyt, doc_col='pub_date', 
                                  hit_col='pub_dateonly', 
                pattern=r"(201[2-6]-[01][0-9]-[0-3][0-9])T", 
                out_type='string', replace=False)

## Join on post/pub dates and other fields

In [234]:
df_name_matches3 = pd.merge(left=df.loc[(df['matched_on'].isna()) &
                                       (df['post_date'] != '') &
                                       (df['post_date'].isna()==False)], 
                           right=df_nyt, 
                           how='inner', 
                           left_on=['name', 'post_date'], 
                           right_on=['main_headline', 'pub_dateonly'],
                           suffixes=('_nyt', '_nyt'))
len(df_name_matches3)

17

In [235]:
# Check if more than one NYT article matched with a single facebook link
print(len(df_name_matches3[df_name_matches3.duplicated(subset=['id'])]))
df_name_matches3[df_name_matches3.duplicated(subset=['id'], keep=False)]

1


,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,news_desk,section_name,byline,type_of_material,_id,word_count,uri,main_headline,link_date_nyt,pub_dateonly
13,5281959998_10150736117549999,5281959998,Unpublished Black History,Hundreds of images from black history have lon...,"Revealing moments in black history, with unpub...",nytimes.com,link,shared_story,2737,60,...,National,U.S.,"{'original': 'By The New York Times', 'person'...",News,nyt://article/178c109b-562d-57e2-b80b-3bdd597d...,71,nyt://article/178c109b-562d-57e2-b80b-3bdd597d...,Unpublished Black History,2016/01/31,2016-02-01
14,5281959998_10150736117549999,5281959998,Unpublished Black History,Hundreds of images from black history have lon...,"Revealing moments in black history, with unpub...",nytimes.com,link,shared_story,2737,60,...,The Upshot,The Upshot,"{'original': '', 'person': [], 'organization':...",Slideshow,nyt://slideshow/82633b8e-bfdf-5fc9-a909-77e9a4...,0,nyt://slideshow/82633b8e-bfdf-5fc9-a909-77e9a4...,Unpublished Black History,2016/01/31,2016-02-01


Looks like duplicates based on different types of articles.

In [236]:
# Check for duplicate NYT articles that were matched with Facebook posts
print(len(df_name_matches3[df_name_matches3.duplicated(subset=['_id'])]))
df_name_matches3[df_name_matches3.duplicated(subset=['_id'], keep=False)]

6


,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,news_desk,section_name,byline,type_of_material,_id,word_count,uri,main_headline,link_date_nyt,pub_dateonly
0,5281959998_422066364533996,5281959998,Snow Fall: The Avalanche at Tunnel Creek,"On Wednesday, here in our Facebook feed, we pu...",The first passage of New York Times reporter J...,projects.nytimes.com,link,shared_story,231,14,...,Sports,Sports,"{'original': 'By John Branch', 'person': [{'fi...",News,nyt://article/e6f507b9-d8c1-5d3e-bdd6-8f591cd9...,16574,nyt://article/e6f507b9-d8c1-5d3e-bdd6-8f591cd9...,Snow Fall: The Avalanche at Tunnel Creek,2012/12/21,2012-12-20
1,5281959998_497482543629133,5281959998,Snow Fall: The Avalanche at Tunnel Creek,Special ReportA group of world-class skiers an...,Fresh powder beckoned 16 expert skiers and sno...,nytimes.com,link,shared_story,538,64,...,Sports,Sports,"{'original': 'By John Branch', 'person': [{'fi...",News,nyt://article/e6f507b9-d8c1-5d3e-bdd6-8f591cd9...,16574,nyt://article/e6f507b9-d8c1-5d3e-bdd6-8f591cd9...,Snow Fall: The Avalanche at Tunnel Creek,2012/12/21,2012-12-20
4,5281959998_10150519725889999,5281959998,Bartender Accused of Threatening to Poison Boe...,A bartender who worked at Speaker John A. Boeh...,A bartender who worked at Speaker John A. Boeh...,nytimes.com,link,shared_story,41,8,...,,U.S.,"{'original': 'By Michael S. Schmidt', 'person'...",News,nyt://article/7c8c24f9-b974-5d7f-ab16-f76fb0d4...,450,nyt://article/7c8c24f9-b974-5d7f-ab16-f76fb0d4...,Bartender Accused of Threatening to Poison Boe...,2015/01/13,2015-01-14
5,5281959998_10150519727084999,5281959998,Bartender Accused of Threatening to Poison Boe...,A bartender who worked at Speaker John A. Boeh...,A bartender who worked at Speaker John A. Boeh...,nytimes.com,link,shared_story,38,7,...,,U.S.,"{'original': 'By Michael S. Schmidt', 'person'...",News,nyt://article/7c8c24f9-b974-5d7f-ab16-f76fb0d4...,450,nyt://article/7c8c24f9-b974-5d7f-ab16-f76fb0d4...,Bartender Accused of Threatening to Poison Boe...,2015/01/13,2015-01-14
6,5281959998_10150542511689999,5281959998,Netanyahu Addresses Congress on Iran,See the updates from Prime Minister Benjamin N...,Prime Minister Benjamin Netanyahu of Israel de...,nyti.ms,link,shared_story,278,96,...,World,World,"{'original': '', 'person': [], 'organization':...",Slideshow,nyt://slideshow/69e0fc87-9658-5d93-b461-72d18d...,0,nyt://slideshow/69e0fc87-9658-5d93-b461-72d18d...,Netanyahu Addresses Congress on Iran,2015/03/03,2015-03-03
7,5281959998_10150542519869999,5281959998,Netanyahu Addresses Congress on Iran,Prime Minister Benjamin Netanyahu - בנימין נתנ...,Prime Minister Benjamin Netanyahu of Israel de...,nytimes.com,link,shared_story,185,59,...,World,World,"{'original': '', 'person': [], 'organization':...",Slideshow,nyt://slideshow/69e0fc87-9658-5d93-b461-72d18d...,0,nyt://slideshow/69e0fc87-9658-5d93-b461-72d18d...,Netanyahu Addresses Congress on Iran,2015/03/03,2015-03-03
8,5281959998_10150542524539999,5281959998,Netanyahu Addresses Congress on Iran,Prime Minister Benjamin Netanyahu - בנימין נתנ...,Prime Minister Benjamin Netanyahu of Israel de...,nytimes.com,link,shared_story,206,56,...,World,World,"{'original': '', 'person': [], 'organization':...",Slideshow,nyt://slideshow/69e0fc87-9658-5d93-b461-72d18d...,0,nyt://slideshow/69e0fc87-9658-5d93-b461-72d18d...,Netanyahu Addresses Congress on Iran,2015/03/03,2015-03-03
9,5281959998_10150542527644999,5281959998,Netanyahu Addresses Congress on Iran,“At a time when many hope that Iran will join ...,Prime Minister Benjamin Netanyahu of Israel de...,nytimes.com,link,shared_story,264,71,...,World,World,"{'original': '', 'person': [], 'organization':...",Slideshow,nyt://slideshow/69e0fc87-9658-5d93-b461-72d18d...,0,nyt://slideshow/69e0fc87-9658-5d93-b461-72d18d...,Netanyahu Addresses Congress on Iran,2015/03/03,2015-03-03
10,5281959998_10150542532199999,5281959998,Netanyahu Addresses Congress on Iran,'If Iran wants to be tre

In [237]:
# update which facebook posts have been joined up, so can run further joins
# on different criteria, and only on those that aren't yet matched
df.loc[df['id'].isin(df_name_matches3['id'].values), 'matched_on'] = 'name'

In [238]:
df_desc_matches3 = pd.merge(left=df.loc[(df['matched_on'].isna()) &
                                       (df['post_date'] != '') &
                                       (df['post_date'].isna()==False)], 
                           right=df_nyt, 
                           how='inner', 
                           left_on=['description', 'post_date'], 
                           right_on=['snippet', 'pub_dateonly'],
                           suffixes=('_nyt', '_nyt'))
len(df_desc_matches3)

1

In [239]:
# Check if more than one NYT article matched with a single facebook link
print(len(df_desc_matches3[df_desc_matches3.duplicated(subset=['id'])]))
df_desc_matches3[df_desc_matches3.duplicated(subset=['id'], keep=False)]

0


,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,news_desk,section_name,byline,type_of_material,_id,word_count,uri,main_headline,link_date_nyt,pub_dateonly


In [240]:
# Check for duplicate NYT articles that were matched with Facebook posts
print(len(df_desc_matches3[df_desc_matches3.duplicated(subset=['_id'])]))
df_desc_matches3[df_desc_matches3.duplicated(subset=['_id'], keep=False)]

0


,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,news_desk,section_name,byline,type_of_material,_id,word_count,uri,main_headline,link_date_nyt,pub_dateonly


In [241]:
# update which facebook posts have been joined up, so can run further joins
# on different criteria, and only on those that aren't yet matched
df.loc[df['id'].isin(df_desc_matches3['id'].values), 'matched_on'] = 'desc'

## Assessment of remaining unmatched posts

In [244]:
df.loc[df['matched_on'].isna()]

,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,trim_link,dupes_on_link,dupe_link_child,dupes_on_desc,dupe_desc_child,dupes_on_name,dupe_name_child,matched_on,link_date,post_date
2,5281959998_467433813289503,5281959998,The Organic Fable,"'Organic has long since become an ideology, th...",A study exposes the hype behind a pseudoscient...,nytimes.com,link,shared_story,812,290,...,https://www.nytimes.com/2012/09/07/opinion/rog...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012/09/07,2012-09-08
3,5281959998_10150252974589999,5281959998,Timeline Photos,Our special Arts & Leisure section 'The New Se...,NaN,NaN,photo,added_photos,513,17,...,https://www.facebook.com/nytimes/photos/a.2835...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,,2012-09-08
14,5281959998_10150253207659999,5281959998,Timeline Photos,Serena Williams defeated Victoria Azarenka to ...,NaN,NaN,photo,added_photos,7629,353,...,https://archive.nytimes.com/www.nytimes.com/in...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,2012/09/08,2012-09-09
26,5281959998_269424823169279,5281959998,Jessica Chastain TimesTalks Live Webcast,NaN,Watch the free livestream here: http://nyti.ms...,Jessica Chastain TimesTalks Live Webcast,event,created_event,51,7,...,https://www.facebook.com/events/269424823169279/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,2012-09-10
27,5281959998_10150253426994999,5281959998,Timeline Photos,Follow live updates from the U.S. Open Men's F...,NaN,NaN,photo,added_photos,467,93,...,https://archive.nytimes.com/www.nytimes.com/in...,1.0,NaN,NaN,NaN,1.0,1.0,NaN,2012/09/10,2012-09-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47845,5281959998_10150958061204999,5281959998,Nicholas Kristof,"Nicholas Kristof, a columnist for The New York...",Our last Facebook Live before the election! Ch...,NaN,video,shared_story,144,40,...,https://www.facebook.com/kristof/videos/101545...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,,2016-11-07
47847,5281959998_10150958085764999,5281959998,The New York Times - Well - Health,Lauren Hard and Paul Moon of The New York Time...,"We're at Pure Yoga with Sarah Attia, founder o...",NaN,video,shared_story,70,6,...,https://www.facebook.com/WellNYT/videos/116561...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,,2016-11-07
47848,5281959998_10150958024099999,5281959998,Whole-Roasted Stuffed Squash Recipe,A vegetarian main to rival every other dish at...,Here is a vegetarian dinner course of impressi...,cooking.nytimes.com,link,shared_story,277,52,...,https://cooking.nytimes.com/recipes/1017780-wh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,2016-11-07
47859,5281959998_10150958344969999,5281959998,2016 Election Puppet Show,We're putting on a live puppet show re-enactin...,NaN,NaN,video,added_video,1211,802,...,https://www.facebook.com/nytimes/videos/101509...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,2016-11-07


In [300]:
df.loc[[2]].values

array([['5281959998_467433813289503', 5281959998, 'The Organic Fable',
        "'Organic has long since become an ideology, the romantic back-to-nature obsession of an upper middle class able to afford it and oblivious, in their affluent narcissism, to the challenge of feeding a planet whose population will surge to 9 billion,' writes Op-Ed columnist Roger Cohen.",
        'A study exposes the hype behind a pseudoscientific ideology. The future is nonorganic.',
        'nytimes.com', 'link', 'shared_story', 812, 290, 421, 0, 0, 0, 0,
        0, 0,
        'http://www.nytimes.com/2012/09/07/opinion/roger-cohen-the-organic-fable.html',
        'https://external.xx.fbcdn.net/safe_image.php?d=AQA6AciF5lLAnN7B&w=130&h=130&url=http%3A%2F%2Fgraphics8.nytimes.com%2Fimages%2F2012%2F09%2F06%2Fopinion%2Fiht-edcohen07-illo%2Fiht-edcohen07-illo-thumbLarge.jpg&cfs=1',
        '2012-09-08 16:44:02', 1.0, nan,
        'http://www.nytimes.com/2012/09/07/opinion/roger-cohen-the-organic-fable.html',
    

In [260]:
# Pull out root from link; maybe that will help shed light on a pattern in 
# the unmatched
df = nlp_prep.pattern_match_in_df(df, doc_col='trim_link', 
                                  hit_col='link_root', 
                pattern=r"(https://www.[a-zA-Z0-9/.-]+.com/[^/]+)", 
                out_type='string', replace=False)

In [261]:
df.loc[df['link_root'].isna(), 'trim_link'].value_counts()

Series([], Name: trim_link, dtype: int64)

In [262]:
df.loc[df['matched_on'].isna(), 'link_root'].value_counts()[:25]

 https://www.facebook.com/nytimes            2097
                                             1398
 https://www.nytimes.com/interactive          307
 https://www.nytimes.com/live                 299
 https://www.facebook.com/nytvideo            141
 https://www.nytimes.com/2014                 113
 https://www.nytimes.com/2015                  88
 https://www.youtube.com/watch                 87
 https://www.nytimes.com/aponline              79
 https://www.nytimes.com/newsgraphics          79
 https://www.nytimes.com/video                 59
 https://www.nytimes.com/projects              52
 https://www.instagram.com/p                   52
 https://www.facebook.com/nytimesstyles        48
 https://www.nytimes.com/elections             43
 https://www.facebook.com/nytopinion           40
 https://www.nytimes.com/politics              38
 https://www.facebook.com/nytimesscience       33
 https://www.nytimes.com/2016                  29
 https://www.facebook.com/NewYorkTodayNYT      27


In [277]:
df.loc[(df['link_root']==' https://www.nytimes.com/interactive') & 
       (df['matched_on'].isna()), 
       ['trim_link', 'link_root', 'matched_on']][:5].values

array([['https://www.nytimes.com/interactive/2012/10/10/dining/the-story-of-a-freshdirect-order.html',
        ' https://www.nytimes.com/interactive', nan],
       ['https://www.nytimes.com/interactive/2012/11/06/us/politics/instagramming-the-election.html',
        ' https://www.nytimes.com/interactive', nan],
       ['https://www.nytimes.com/interactive/projects/qa/events/thanksgiving-help-line',
        ' https://www.nytimes.com/interactive', nan],
       ['https://www.nytimes.com/interactive/projects/qa/events/thanksgiving-help-line',
        ' https://www.nytimes.com/interactive', nan],
       ['https://www.nytimes.com/interactive/projects/qa/events/thanksgiving-help-line',
        ' https://www.nytimes.com/interactive', nan]], dtype=object)

In [275]:
df_nyt.loc[df_nyt['web_url'].str.contains("freshdirect-order", 
                                          regex=False)]

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,headline,keywords,...,news_desk,section_name,byline,type_of_material,_id,word_count,uri,main_headline,link_date,pub_dateonly


Perhaps not all of the interactives were returned when I queried the articles API. This page definitely exists in the archive, but I don't see it in the articles list I downloaded form the API.

In [282]:
df.loc[(df['link_root']==' https://www.nytimes.com/2014') & 
       (df['matched_on'].isna()), 
       ['trim_link', 'link_root', 'matched_on']][:15].values

array([['https://www.nytimes.com/2014/01/05/fashion/Phone-Call-Work-From-Home.html/',
        ' https://www.nytimes.com/2014', nan],
       ['https://www.nytimes.com/2014/01/21/your-money/are-you-amending-a-tax-return.html#commentsContainer',
        ' https://www.nytimes.com/2014', nan],
       ['https://www.nytimes.com/2014/01/21/opinion/cohen-twitter-bashing-bores.html',
        ' https://www.nytimes.com/2014', nan],
       ['https://www.nytimes.com/2014/02/06/us/politics/NULL',
        ' https://www.nytimes.com/2014', nan],
       ['https://www.nytimes.com/2014/02/11/world/europe/awaiting-russias-next-move-in-ukraine.html',
        ' https://www.nytimes.com/2014', nan],
       ['https://www.nytimes.com/2014/02/13/opinion/saakashvili-czar-vladimirs-illusions.html',
        ' https://www.nytimes.com/2014', nan],
       ['https://www.nytimes.com/2014/03/11/opinion/the-kremlins-social-media-takeover.html',
        ' https://www.nytimes.com/2014', nan],
       ['https://www.nytimes.com/

In [284]:
df_nyt.loc[df_nyt['web_url'].str.contains("at-pistorius-trial-twitterati", 
                                          regex=False)].values

array([], shape=(0, 22), dtype=object)

Its possible that I may be able to get more matches using fuzzy matching on the name and headline. But some of these I think just aren't included in the NYT articles API, and fuzzy matching may produce a match that isn't accurate.

The final thing I can try is to take the name values and use the NYT article search to search for matches on headline.

# Search Articles for Unmatched

In [316]:
df.loc[(df['matched_on'].isna()) & (df['trim_link'].str.contains('nytimes.com', 
                regex=False))]

,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,dupes_on_link,dupe_link_child,dupes_on_desc,dupe_desc_child,dupes_on_name,dupe_name_child,matched_on,link_date,post_date,link_root
2,5281959998_467433813289503,5281959998,The Organic Fable,"'Organic has long since become an ideology, th...",A study exposes the hype behind a pseudoscient...,nytimes.com,link,shared_story,812,290,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012/09/07,2012-09-08,https://www.nytimes.com/2012
14,5281959998_10150253207659999,5281959998,Timeline Photos,Serena Williams defeated Victoria Azarenka to ...,NaN,NaN,photo,added_photos,7629,353,...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,2012/09/08,2012-09-09,
27,5281959998_10150253426994999,5281959998,Timeline Photos,Follow live updates from the U.S. Open Men's F...,NaN,NaN,photo,added_photos,467,93,...,1.0,NaN,NaN,NaN,1.0,1.0,NaN,2012/09/10,2012-09-10,
29,5281959998_10150253458449999,5281959998,Timeline Photos,Andy Murray defeated Novak Djokovic to win the...,NaN,NaN,photo,added_photos,4324,193,...,1.0,1.0,NaN,NaN,1.0,1.0,NaN,2012/09/10,2012-09-11,
105,5281959998_224679674326401,5281959998,Video: Patriot Game,'POWER YOUR PRESIDENT TO WINNING. USE MANY GOO...,The Gregory Brothers present a video game-insp...,nytimes.com,link,shared_story,178,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012/09/16,2012-09-17,https://www.nytimes.com/video
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47787,5281959998_1301608653216932,5281959998,How to Roast Brussels Sprouts,If you haven’t yet figured out a go-to recipe ...,NaN,NaN,video,added_video,2986,571,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,2016-11-06,
47792,5281959998_10150956594114999,5281959998,Timeline Photos,Ronald Reagan ended up winning 49 states in a ...,NaN,NaN,photo,added_photos,654,192,...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,,2016-11-06,
47806,5281959998_10150953848459999,5281959998,Pasta e Fagioli Recipe,A cozy classic. (via The New York Times Food),This is a classic Italian bean and pasta soup ...,cooking.nytimes.com,link,shared_story,587,54,...,1.0,1.0,1.0,1.0,1.0,1.0,NaN,,2016-11-06,
47848,5281959998_10150958024099999,5281959998,Whole-Roasted Stuffed Squash Recipe,A vegetarian main to rival every other dish at...,Here is a vegetarian dinner course of impressi...,cooking.nytimes.com,link,shared_story,277,52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,2016-11-07,


In [329]:
error_ids = ["5281959998_400268756705284","5281959998_482528545120216","5281959998_10150263192939999","5281959998_10150264225739999","5281959998_10150264240524999","5281959998_348613538571828","5281959998_10150315749014999","5281959998_10150319739834999","5281959998_10150365840569999","5281959998_10150365869739999","5281959998_10150370971784999","5281959998_10150376819204999","5281959998_10150380041949999","5281959998_10150381291804999","5281959998_10150387417999999","5281959998_10102898859763603","5281959998_10102898871410263","5281959998_10150413655074999","5281959998_10150416729024999","5281959998_10150434473354999","5281959998_10150462809124999","5281959998_10150470716529999","5281959998_10150472844929999","5281959998_10150475754474999","5281959998_10150478592399999","5281959998_10150484512369999","5281959998_10150501037409999","5281959998_10150508217624999","5281959998_10150516665944999","5281959998_10150517535969999","5281959998_10150575819834999","5281959998_10150575845844999","5281959998_10150693441289999","5281959998_10150728410104999","5281959998_10150746644634999","5281959998_10150820198739999","5281959998_10150820206884999","5281959998_10150820372214999","5281959998_10150820213294999","5281959998_10150820214794999","5281959998_10150850434459999","5281959998_10150934874619999","5281959998_10150934898589999"]

In [336]:
# I had a handful of links that have # references, so need to take out whatever
# comes after the #'s'
df['trim_link'] = df['trim_link'].map(
    lambda x: re.search(r"^([^#]+)", x).group(1) 
                if (type(x)==str and re.search(r"^([^#]+)", x)) else '')

In [337]:
df.loc[df['id'].isin(error_ids), 'trim_link'].values

array(['https://elections.nytimes.com/2012/debates/vice-presidential/2012-10-11',
       'https://archive.nytimes.com/www.nytimes.com/interactive/us/politics/newspaper-presidential-endorsements.html',
       'https://archive.nytimes.com/www.nytimes.com/interactive/2012/10/28/nyregion/hurricane-sandy.html',
       'https://archive.nytimes.com/www.nytimes.com/interactive/2012/11/06/us/politics/live-election-results.html',
       'https://www.nytimes.com/elections/2012/results/live-coverage.html',
       'https://www.nytimes.com/slideshow/2013/03/09/opinion/sunday/20130310_EXPOSURES-ss.html',
       'https://projects.nytimes.com/live-dashboard/2013-06-26-supreme-court-gay-marriage',
       'https://www.nytimes.com/newsgraphics/2013/quiet/index.html',
       'https://www.nytimes.com/projects/2013/holiday-gift-guide/index.html',
       'https://www.nytimes.com/projects/2013/invisible-child/index.html',
       'https://www.nytimes.com/projects/2013/benghazi/index.html',
       'https://www.n

In [361]:
base_url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json'
count = 0
hits_2 = []
    
# for i in df.loc[(df['matched_on'].isna()) & (df['trim_link'].str.contains(
#     'nytimes.com', regex=False))].index:
# for i in df.loc[df['id'].isin(['5281959998_10150376819204999'])].index:
for i in df.loc[df['id'].isin(ids)].index:
    s_url = df.at[i, 'trim_link']

    url = f"""{base_url}?fq=web_url:("{s_url}")&api-key={nyt_token['api_key']}"""
    response = requests.get(url)
    resp_dict=json.loads(response.text)
    
    count += 1
    
    if 'fault' in resp_dict.keys():
        print(f"Error id {df.at[i, 'id']}")
    else:
        if resp_dict['response']['meta']['hits'] > 0:
            hits_2.append({'id':df.at[i, 'id'], 'hits': resp_dict['response']['docs']})
        
    if count % 500 == 0:
        print(f"Queried {count} urls")
    
    # sleep for 7 seconds since the limit is 10 per minute
    # stopped this because even one request tended to take about 7-8 seconds
    time.sleep(6)
    
with open(f'data/article_url_search_hits2.json', 'w') as f:
    json.dump(hits_2, f)
    f.close()
    

Queried 500 urls
Queried 1000 urls


In [362]:
len(hits_2)

1040

In [371]:
hits_2[0]['hits'][0].keys()

dict_keys(['abstract', 'web_url', 'snippet', 'lead_paragraph', 'source', 'multimedia', 'headline', 'keywords', 'pub_date', 'document_type', 'news_desk', 'section_name', 'byline', 'type_of_material', '_id', 'word_count', 'uri'])

In [379]:
# pull out first hit with ID to make a dataframe
first_hits = []

for hit in hits_2:
    row = {'id': hit['id']}
    row.update(hit['hits'][0])
    first_hits.append(row)
    
first_hits[0]

{'id': '5281959998_467433813289503',
 'abstract': 'A study exposes the hype behind a pseudoscientific ideology. The future is nonorganic.',
 'web_url': 'https://www.nytimes.com/2012/09/07/opinion/roger-cohen-the-organic-fable.html',
 'snippet': 'A study exposes the hype behind a pseudoscientific ideology. The future is nonorganic.',
 'lead_paragraph': 'LONDON — At some point — perhaps it was gazing at a Le Pain Quotidien menu offering an “organic baker’s basket served with organic butter, organic jam and organic spread” as well as seasonally organic orange juice — I found I just could not stomach the “O” word or what it stood for any longer.',
 'source': 'International Herald Tribune',
 'multimedia': [{'rank': 0,
   'subtype': 'xlarge',
   'caption': None,
   'credit': None,
   'type': 'image',
   'url': 'images/2014/11/01/opinion/cohen-circular/cohen-circular-articleLarge-v2.png',
   'height': 300,
   'width': 600,
   'legacy': {'xlarge': 'images/2014/11/01/opinion/cohen-circular/cohe

In [381]:
df_firsthits = pd.DataFrame(first_hits)
df_firsthits.head()

,id,abstract,web_url,snippet,lead_paragraph,source,multimedia,headline,keywords,pub_date,...,section_name,byline,type_of_material,_id,word_count,uri,slideshow_credits,subsection_name,print_section,print_page
0,5281959998_467433813289503,A study exposes the hype behind a pseudoscient...,https://www.nytimes.com/2012/09/07/opinion/rog...,A study exposes the hype behind a pseudoscient...,LONDON — At some point — perhaps it was gazing...,International Herald Tribune,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","{'main': 'The Organic Fable', 'kicker': 'Op-Ed...","[{'name': 'subject', 'value': 'Food', 'rank': ...",2012-09-06T12:14:52+0000,...,Opinion,"{'original': 'By Roger Cohen', 'person': [{'fi...",Op-Ed,nyt://article/50df30c5-86c9-519c-8f9f-94ab88cd...,866,nyt://article/50df30c5-86c9-519c-8f9f-94ab88cd...,NaN,NaN,NaN,NaN
1,5281959998_417695201626695,Festive and familiar dishes for celebrating th...,https://www.nytimes.com/slideshow/2011/09/21/d...,Festive and familiar dishes for celebrating th...,Festive and familiar dishes for celebrating th...,The New York Times,"[{'rank': 0, 'subtype': 'wide', 'caption': Non...","{'main': 'Recipes for Rosh Hashana', 'kicker':...","[{'name': 'subject', 'value': 'Rosh Hashanah',...",2011-09-20T22:56:05+0000,...,Food,"{'original': None, 'person': [], 'organization...",Slideshow,nyt://slideshow/e8db9197-4c69-51d9-824b-afe278...,0,nyt://slideshow/e8db9197-4c69-51d9-824b-afe278...,Ryan Collerd for The New York Times,NaN,NaN,NaN
2,5281959998_106658229491567,A spokesman said a fight among employees on Su...,https://www.nytimes.com/2012/09/25/technology/...,A spokesman said a fight among employees on Su...,"SHANGHAI — Foxconn Technology, a major supplie...",International Herald Tribune,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","{'main': 'Foxconn Plant Closed After Riot, Com...","[{'name': 'organizations', 'value': 'Foxconn T...",2012-09-24T07:22:51+0000,...,Technology,{'original': 'By David Barboza and Keith Brads...,News,nyt://article/d7e19fed-75a7-586d-9700-e8ae4cfd...,681,nyt://article/d7e19fed-75a7-586d-9700-e8ae4cfd...,NaN,NaN,NaN,NaN
3,5281959998_10150256227144999,NYT: Restaurant Update for 1/20/2006. Topic di...,https://www.nytimes.com/,NYT: Restaurant Update for 1/20/2006. Topic di...,NYT: Restaurant Update for 1/20/2006. Topic di...,WQXR,[],{'main': 'NYT: Restaurant Update for 1/20/2006...,[],2018-08-28T19:42:47+0000,...,Food,"{'original': None, 'person': [], 'organization...",NaN,nyt://audio/00144de5-b091-5f0f-b30c-85c848666b0a,0,nyt://audio/00144de5-b091-5f0f-b30c-85c848666b0a,NaN,NaN,NaN,NaN
4,5281959998_10150261255114999,NYT: Restaurant Update for 1/20/2006. Topic di...,https://www.nytimes.com/,NYT: Restaurant Update for 1/20/2006. Topic di...,NYT: Restaurant Update for 1/20/2006. Topic di...,WQXR,[],{'main': 'NYT: Restaurant Update for 1/20/2006...,[],2018-08-28T19:42:47+0000,...,Food,"{'original': None, 'person': [], 'organization...",NaN,nyt://audio/00144de5-b091-5f0f-b30c-85c848666b0a,0,nyt://audio/00144de5-b091-5f0f-b30c-85c848666b0a,NaN,NaN,NaN,NaN


In [404]:
# make first matches resemble the structure of df_nyt

# pull out main headline
df_firsthits['main_headline'] = df_firsthits[
    'headline'].map(lambda x: x['main'])

In [405]:
# drop extra columns
df_firsthits.drop(columns=['slideshow_credits', 'subsection_name'], 
                           inplace=True)

In [407]:
# Join first hits to facebook posts to assess
df_linksearch_matches = pd.merge(left=df.loc[(df['matched_on'].isna())], 
                           right=df_firsthits, 
                           how='inner', 
                           left_on='id', 
                           right_on='id',
                           suffixes=('_orig', '_new'))
len(df_linksearch_matches)

1040

In [409]:
df_linksearch_matches[['id', 'name', 'main_headline', 'trim_link', 'web_url', 
                       'posted_at', 'pub_date', 'description', 'snippet']]

,id,name,main_headline,trim_link,web_url,posted_at,pub_date,description,snippet
0,5281959998_467433813289503,The Organic Fable,The Organic Fable,https://www.nytimes.com/2012/09/07/opinion/rog...,https://www.nytimes.com/2012/09/07/opinion/rog...,2012-09-08 16:44:02,2012-09-06T12:14:52+0000,A study exposes the hype behind a pseudoscient...,A study exposes the hype behind a pseudoscient...
1,5281959998_417695201626695,The New York Times Food,Recipes for Rosh Hashana,https://www.nytimes.com/slideshow/2011/09/21/d...,https://www.nytimes.com/slideshow/2011/09/21/d...,2012-09-17 20:46:12,2011-09-20T22:56:05+0000,Chocolate-Apricot Babka: http://nyti.ms/RZVv0F...,Festive and familiar dishes for celebrating th...
2,5281959998_106658229491567,Foxconn Plant in China Closed After Worker Riot,"Foxconn Plant Closed After Riot, Company Says",https://www.nytimes.com/2012/09/25/technology/...,https://www.nytimes.com/2012/09/25/technology/...,2012-09-24 16:44:29,2012-09-24T07:22:51+0000,A spokesman said a fight among employees on Su...,A spokesman said a fight among employees on Su...
3,5281959998_10150256227144999,NaN,NYT: Restaurant Update for 1/20/2006,https://www.nytimes.com/,https://www.nytimes.com/,2012-09-27 04:02:02,2018-08-28T19:42:47+0000,NaN,NYT: Restaurant Update for 1/20/2006. Topic di...
4,5281959998_10150261255114999,NaN,NYT: Restaurant Update for 1/20/2006,https://www.nytimes.com/,https://www.nytimes.com/,2012-10-23 03:49:24,2018-08-28T19:42:47+0000,NaN,NYT: Restaurant Update for 1/20/2006. Topic di...
...,...,...,...,...,...,...,...,...,...
1035,5281959998_10150954475304999,Pumpkin Bread With Brown Butter and Bourbon Re...,Pumpkin Bread With Brown Butter and Bourbon,https://cooking.nytimes.com/recipes/1017797-pu...,https://cooking.nytimes.com/recipes/1017797-pu...,2016-11-05 09:55:01,2015-11-18T00:00:00+0000,This hearty pumpkin bread is a sophisticated t...,
1036,5281959998_10150953811444999,Mashed Potato Casserole Recipe,Mashed Potato Casserole,https://cooking.nytimes.com/recipes/1013923-ma...,https://cooking.nytimes.com/recipes/1013923-ma...,2016-11-05 13:55:00,2011-11-16T00:00:00+0000,This casserole may upset some mashed potato pu...,
1037,5281959998_1301608653216932,How to Roast Brussels Sprouts,Roasted Brussels Sprouts With Garlic,https://cooking.nytimes.com/recipes/1890-roast...,https://cooking.nytimes.com/recipes/1890-roast...,2016-11-06 16:55:00,2005-12-07T00:00:00+0000,NaN,
1038,5281959998_10150953848459999,Pasta e Fagioli Recipe,Pasta e Fagioli,https://cooking.nytimes.com/recipes/1016031-pa...,https://cooking.nytimes.com/recipes/1016031-pa...,2016-11-06 23:55:00,2014-02-11T00:00:00+0000,This is a classic Italian bean and pasta soup ...,


In [410]:
df_linksearch_matches[['id', 'name', 'main_headline', 'trim_link', 'web_url', 
                       'posted_at', 'pub_date', 'description', 'snippet']].\
to_csv('data/linksearch_matches.csv')

I reviewed the matches separately outside this notebook, and found them to be good except for some Timeline Photos posts that matched based on a link to the NYT homepage, which doesn't look right. I'm going to remove those. 

The vast majority of links found using this method were recipes. I suspect recipes were not included in the initial article metadata export, or else there were also instances of recipes being linked to way after they had been published, and the initial article metadata search was done based on published date.

In [411]:
to_drop = [3,4,5,6,7,8,10,11,12,13,15,20,21,24,26,27,28,29,30,31,34,35,36,38,
           41,42,69,179,184,187,189,196,198,200,204,206,248,397,606]

df_linksearch_matches.drop(index=to_drop, inplace=True)

In [413]:
df_linksearch_matches.head()

,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,news_desk,section_name,byline,type_of_material,_id,word_count,uri,print_section,print_page,main_headline
0,5281959998_467433813289503,5281959998,The Organic Fable,"'Organic has long since become an ideology, th...",A study exposes the hype behind a pseudoscient...,nytimes.com,link,shared_story,812,290,...,Editorial,Opinion,"{'original': 'By Roger Cohen', 'person': [{'fi...",Op-Ed,nyt://article/50df30c5-86c9-519c-8f9f-94ab88cd...,866,nyt://article/50df30c5-86c9-519c-8f9f-94ab88cd...,NaN,NaN,The Organic Fable
1,5281959998_417695201626695,5281959998,The New York Times Food,"If you're celebrating Rosh Hashana, what was, ...",Chocolate-Apricot Babka: http://nyti.ms/RZVv0F...,NaN,photo,shared_story,923,45,...,Food,Food,"{'original': None, 'person': [], 'organization...",Slideshow,nyt://slideshow/e8db9197-4c69-51d9-824b-afe278...,0,nyt://slideshow/e8db9197-4c69-51d9-824b-afe278...,NaN,NaN,Recipes for Rosh Hashana
2,5281959998_106658229491567,5281959998,Foxconn Plant in China Closed After Worker Riot,"Foxconn Technology, a major supplier to some o...",A spokesman said a fight among employees on Su...,nytimes.com,link,shared_story,124,22,...,Business,Technology,{'original': 'By David Barboza and Keith Brads...,News,nyt://article/d7e19fed-75a7-586d-9700-e8ae4cfd...,681,nyt://article/d7e19fed-75a7-586d-9700-e8ae4cfd...,NaN,NaN,"Foxconn Plant Closed After Riot, Company Says"
9,5281959998_296055523846833,5281959998,Thanks for Not Sharing,“There is a new urge to behave as if life were...,There is a new urge to behave as if life were ...,nytimes.com,link,shared_story,692,70,...,Editorial,Opinion,"{'original': 'By Roger Cohen', 'person': [{'fi...",Op-Ed,nyt://article/9413c573-5b13-53c4-bae4-9223a0c7...,850,nyt://article/9413c573-5b13-53c4-bae4-9223a0c7...,NaN,NaN,Thanks for Not Sharing
14,5281959998_10150276934359999,5281959998,Timeline Photos,The affinity that cauliflower has with black o...,NaN,NaN,photo,added_photos,2061,100,...,,NaN,"{'original': 'Martha Rose Shulman', 'person': ...",Recipe,nyt://recipe/383cbf04-9dec-5ad8-a653-1fd686b42631,0,nyt://recipe/383cbf04-9dec-5ad8-a653-1fd686b42631,NaN,NaN,Sicilian Cauliflower and Black Olive Gratin


In [412]:
# update which facebook posts have been joined up, so can run further joins
# on different criteria, and only on those that aren't yet matched
df.loc[df['id'].isin(df_linksearch_matches['id'].values), 'matched_on'] = 'link_search'

# Initial Matches Assembly


In [414]:
df_list = [df_link_matches, df_desc_matches, df_name_matches, 
           df_link_matches2, df_desc_matches2, df_name_matches2,
           df_desc_matches3, df_name_matches3, df_linksearch_matches]

df_matches = pd.concat(df_list, axis=0, join='inner', ignore_index=True, )

df_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43448 entries, 0 to 43447
Data columns (total 50 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                43448 non-null  object 
 1   page_id           43448 non-null  int64  
 2   name              42865 non-null  object 
 3   message           41847 non-null  object 
 4   description       41646 non-null  object 
 5   caption           41587 non-null  object 
 6   post_type         43448 non-null  object 
 7   status_type       43448 non-null  object 
 8   likes_count       43448 non-null  int64  
 9   comments_count    43448 non-null  int64  
 10  shares_count      43448 non-null  int64  
 11  love_count        43448 non-null  int64  
 12  wow_count         43448 non-null  int64  
 13  haha_count        43448 non-null  int64  
 14  sad_count         43448 non-null  int64  
 15  thankful_count    43448 non-null  int64  
 16  angry_count       43448 non-null  int64 

In [415]:
match_links3 = df_matches['web_url'].unique()
len(match_links3)

39754

In [416]:
# dump out latest dataframe of matched articles
f = gzip.open('data/matches_backup_7-5-2021.pickle.gz','wb')
pickle.dump(df_matches, f)
f.close()

#df_matches.to_pickle('data/matches_backup_7-1-2021.pickle')

In [181]:
#pd.DataFrame(match_links2).to_csv("data/match_links2.csv")

I exported the list of URLs and ran web scraping using Selenium to get the comments count in a separate notebook.

# Reviewing Duplicate Matches
Specifically, where a single Facebook post matched on more than one NYT article. Is the metadata I want to use significantly different?

In [421]:
df_matches.sort_values('id', inplace=True)

In [422]:
print(len(df_matches[df_matches.duplicated(subset=['id'])]))

df_matches[df_matches.duplicated(subset=['id'], keep=False)]

221


,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,keywords,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri
36604,5281959998_10150299457829999,5281959998,Updates on Aftermath of Boston Marathon Explos...,'Something is wrong. They have been framed.' R...,The Lede is following the aftermath of Monday’...,thelede.blogs.nytimes.com,link,shared_story,342,136,...,"[{'name': 'subject', 'value': 'Boston Marathon...",2013-04-19T10:25:31+0000,article,,Blogs,"{'original': 'By The New York Times', 'person'...",News,nyt://article/c71f2857-d9ff-50c1-9cc7-3baf14e5...,13612,nyt://article/c71f2857-d9ff-50c1-9cc7-3baf14e5...
36603,5281959998_10150299457829999,5281959998,Updates on Aftermath of Boston Marathon Explos...,'Something is wrong. They have been framed.' R...,The Lede is following the aftermath of Monday’...,thelede.blogs.nytimes.com,link,shared_story,342,136,...,"[{'name': 'subject', 'value': 'Boston Marathon...",2013-04-18T14:35:07+0000,article,,Blogs,"{'original': 'By Christine Hauser, Jennifer Pr...",News,nyt://article/8d63a22b-24a5-5527-a7b2-2888674d...,4192,nyt://article/8d63a22b-24a5-5527-a7b2-2888674d...
36625,5281959998_10150306238154999,5281959998,Video: Bill Cunningham | Full Bloom,"Bill Cunningham, inspired by the four-foot-lon...","A focus of fashion is the length of the leg, s...",nytimes.com,link,shared_story,175,3,...,"[{'name': 'subject', 'value': 'Fashion and App...",2013-05-18T02:48:10+0000,multimedia,,Fashion & Style,"{'original': 'Joanna Nikas', 'person': [{'firs...",Video,nyt://video/854ae85f-5a35-5e2c-ba51-107b1567c087,0,nyt://video/854ae85f-5a35-5e2c-ba51-107b1567c087
36626,5281959998_10150306238154999,5281959998,Video: Bill Cunningham | Full Bloom,"Bill Cunningham, inspired by the four-foot-lon...","A focus of fashion is the length of the leg, s...",nytimes.com,link,shared_story,175,3,...,"[{'name': 'subject', 'value': 'Fashion and App...",2013-05-20T16:54:55+0000,article,,Style,"{'original': 'By Bill Cunningham', 'person': [...",News,nyt://article/c4185b01-90be-5fbb-85d8-1f6f7d74...,47,nyt://article/c4185b01-90be-5fbb-85d8-1f6f7d74...
36636,5281959998_10150309977124999,5281959998,Video: Escape From North Korea,'The government doesn't give us a thing but th...,A smuggler named Dragon attempts to guide Nort...,nytimes.com,link,shared_story,386,20,...,"[{'name': 'glocations', 'value': 'North Korea'...",2013-06-06T01:05:08+0000,multimedia,Opinion,Opinion,"{'original': 'Ann Shin', 'person': [{'firstnam...",Video,nyt://video/786833f5-cd66-5444-9cdf-75707417146a,0,nyt://video/786833f5-cd66-5444-9cdf-75707417146a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1498,5281959998_350894045015656,5281959998,Q. and A. on the Papal Transition,White smoke poured from a chimney atop the Sis...,Times reporters covering the papal transition ...,thelede.blogs.nytimes.com,link,shared_story,813,102,...,[],2013-03-11T20:44:42+0000,article,,Blogs,"{'original': 'By THE NEW YORK TIMES', 'person'...",News,nyt://article/8c62f483-ffe2-5091-865c-77d2a0f9...,1915,nyt://article/8c62f483-ffe2-5091-865c-77d2a0f9...
1499,5281959998_350894045015656,5281959998,Q. and A. on the Papal Transition,White smoke poured from a chimney atop the Sis...,Times reporters covering the papal transition ...,thelede.blogs.nytimes.com,link,shared_story,813,102,...,"[{'name': 'subject', 'value': 'Cardinals (Roma...",2013-03-11T20:44:42+0000,article,,Blogs,"{'original': 'By The New York Times', 'person'...",News,nyt://article/c3d1cb31-6132-5868-81ca-911d05c2...,1915,nyt://article/c3d1cb31-6132-5868-81ca-911d05c2...
37568,5281959998_423902177671528,5281959998,Global Citizen Festival,"Beginning at 4:00pm ET, watch the free live we...","On Sept. 29 at 4:30 p.m. EST, watch a live vid...",nytimes.com,link,shared_story,177,10,...,"[{'name': 'subject', 'value': 'Music', 'rank':...",2015-09-27T21:11:14+0000,multimedia,Arts,Arts,"{'original': '', 'p

In [423]:
df_matches[df_matches.duplicated(subset=['id'], keep=False)].to_csv(
'data/review_duplicatematches.csv')

I reviewed these duplicates outside of this notebook, and did my best to identify the best single match from Facebook post to article.

If the Facebook post appeared to be a video and the NYT articles had both an article and a multimedia feature, I chose the multimedia. However, most of the Facebook posts were just links, and didn't specify whether they were links to articles, slideshows, or videos. In those cases, I chose the article link that matched the Facebook post link the best.

There were also a few recurring sets of photos that matched erroneously, and I elected to drop all of these matches as there were no correct ones.

In [424]:
to_drop = [36603,36626,36637,36666,37624,36717,37629,36723,36757,36761,36792,
           37670,37666,37669,37667,37668,37665,37671,36847,40060,40062,36866,
           36950,37799,37793,37792,37795,37791,37796,37797,37794,37798,37800,
           37790,37827,37828,37831,37832,37837,37824,37820,37825,37821,37823,
           37819,37833,37835,37834,37822,37826,37830,37829,37836,37841,37843,
           37842,37840,37839,37844,37845,37854,37849,37855,37851,37856,37852,
           37848,37847,37850,37853,37877,37954,37879,37926,37931,37893,37900,
           37905,37947,37889,37957,37886,37953,37921,37894,37952,37924,37898,
           37942,37945,37935,37919,37891,37950,37912,37888,37941,37860,37869,
           37890,37868,37933,37925,37870,37901,37915,37865,37906,37920,37867,
           37914,37862,37871,37932,37872,37887,37928,37892,37882,37863,37918,
           37908,37866,37902,37913,37861,37899,37936,37922,37911,37939,37873,
           37938,37934,37864,37917,37883,37927,37910,37875,37943,37878,37909,
           37944,37923,37896,37904,37881,37897,37885,37884,37955,37951,37916,
           37956,37946,37940,37876,37949,37929,37903,37907,37880,37948,37930,
           37895,37874,37937,37034,37999,40363,13716,37133,38025,37190,37228,
           37251,17862,41776,18034,41778,37266,18225,18257,18282,18377,18382,
           18385,37278,18404,41810,18458,40511,40513,41896,41895,37321,37339,
           41898,41899,37361,41902,41903,37421,37425,37424,42444,26526,38158,
           38166,38173,42299,42301,37474,37487,42342,42344,38202,38205,38218,
           42384,42386,37632,36735,1498,37567,37566]

df_matches.drop(index=to_drop, inplace=True)

In [430]:
df_matches[df_matches.duplicated(subset=['id'], keep=False)]

,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,...,keywords,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri


In [431]:
len(df_matches)

43221

# Export final matches

In [432]:
# pull out the NYT article metadata fields I will potentiall want to use in
# EDA and modeling

f = gzip.open('data/final_matches.pickle.gz','wb')
pickle.dump(df_matches[['id', '_id', 'print_section', 'print_page', 
            'keywords', 'news_desk', 'section_name', 'type_of_material', 
            'word_count']], f)
f.close()

# Scrape Comments Counts

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import urllib

In [ ]:
# initiate webdriver. This should launch a Chrome window controlled by Selenium
driver = webdriver.Chrome('/Users/jessicamiles/Downloads/chromedriver')

In [ ]:
# get Selenium window to load a NYT article.
## Manually sign in to NYT using my login to get access to subscriber material
driver.get('https://www.nytimes.com/2016/10/31/opinion/strangers-on-a-strange-election.html')

In [ ]:
# Look through links and scrape comments. Save in a dictionary and save the
# whole dictionary to file every 100 links
comments = []
count = 0
dump_ct = 158

for link in match_links1['link'][15800:]:
    
    count +=1
    
    driver.get(link)

    com = driver.find_elements_by_xpath('//*[@id="comments-speech-bubble-top"]')
    
    if len(com) == 2:
        comments.append({'link':link, 'comments':com[1].text, 'archived':0})
    elif len(com) == 1:
        comments.append({'link':link, 'comments':com[0].text, 'archived':0})
    else:
        # check if the url was redirected to an archive. If so, there may have
        # once been comments, but we're accessing an archived version of the
        # page, so we can't see them
        if 'archive.nytimes' in driver.current_url:
            comments.append({'link':link, 'comments':-1, 'archived':1})
        else:
            comments.append({'link':link, 'comments':-1, 'archived':0})
    
    # dump to json every 100 links, but keep a running tally so the last
    # will have everything
    if count==100:
        with open(f'comments_backup_{dump_ct}.json', 'w') as f:
            json.dump(comments, f)
            f.close()

        dump_ct += 1
        count = 0
        
    

# Extra

In [146]:
df_expanded1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9553 entries, 1834 to 26347
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             9553 non-null   object
 1   link_expanded  9553 non-null   object
dtypes: object(2)
memory usage: 223.9+ KB


In [144]:
df_expanded1a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26349 entries, 0 to 26348
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             26349 non-null  object
 1   link_expanded  26349 non-null  object
dtypes: object(2)
memory usage: 411.8+ KB


In [141]:
df_message_links.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2217 entries, 1 to 3135
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             2217 non-null   object
 1   link_expanded  2217 non-null   object
dtypes: object(2)
memory usage: 52.0+ KB


In [149]:
df_expanded = pd.concat([df_expanded1, df_expanded1a, df_message_links], 
                        ignore_index=True)
df_expanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38119 entries, 0 to 38118
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             38119 non-null  object
 1   link_expanded  38119 non-null  object
dtypes: object(2)
memory usage: 595.7+ KB


In [150]:
df_expanded.to_csv('data/expanded_links_all.csv.gz', compression='gzip')

In [134]:
# import links expanded from first batch
df_expanded1 = pd.read_json('data/expanded_links1.json', 
                                dtype='object')
df_expanded1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9553 entries, 1834 to 26347
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             9553 non-null   object
 1   link           9553 non-null   object
 2   link_expanded  9553 non-null   object
dtypes: object(3)
memory usage: 298.5+ KB


In [135]:
df_expanded1.tail()

,id,link,link_expanded
15144,5281959998_10150485781054999,http://nyti.ms/1sVukYB,https://www.nytimes.com/2014/11/16/world/mexic...
15145,5281959998_10150485788419999,http://nyti.ms/1EO0Z8a,https://www.nytimes.com/2014/11/14/nyregion/sy...
15146,5281959998_10150485805269999,http://nyti.ms/11bNEth,https://www.nytimes.com/2014/11/16/business/an...
15147,5281959998_10150485814414999,http://nyti.ms/1uBLdf5,https://www.nytimes.com/2014/11/13/science/kee...
26347,5281959998_10150636148279999,http://nyti.ms/1LWEyUY,https://www.nytimes.com/2015/08/02/sports/golf...


In [17]:
# concatenate with main df and then I want to save to back it up
df = df.join(df_expanded1['link_expanded'], how='left')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47867 entries, 0 to 47866
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              47867 non-null  object
 1   page_id         47867 non-null  int64 
 2   name            46574 non-null  object
 3   message         46132 non-null  object
 4   description     43512 non-null  object
 5   caption         42892 non-null  object
 6   post_type       47867 non-null  object
 7   status_type     47856 non-null  object
 8   likes_count     47867 non-null  int64 
 9   comments_count  47867 non-null  int64 
 10  shares_count    47867 non-null  int64 
 11  love_count      47867 non-null  int64 
 12  wow_count       47867 non-null  int64 
 13  haha_count      47867 non-null  int64 
 14  sad_count       47867 non-null  int64 
 15  thankful_count  47867 non-null  int64 
 16  angry_count     47867 non-null  int64 
 17  link            47201 non-null  object
 18  pictur

In [18]:
df.loc[df['link_expanded'].isna()==False, 
            ['id', 'link', 'link_expanded']]

,id,link,link_expanded
1834,5281959998_298385756956423,http://nyti.ms/YduMRx,https://www.nytimes.com/2013/03/06/opinion/sho...
1837,5281959998_338711676230154,http://nyti.ms/WwFEjm,https://www.nytimes.com/2013/03/07/technology/...
1842,5281959998_256954574440999,http://nyti.ms/ZiJ0Sb,https://www.nytimes.com/2013/03/07/opinion/coa...
1844,5281959998_280841352048945,http://nyti.ms/W8dSHj,https://www.nytimes.com/2013/03/07/fashion/app...
1851,5281959998_519969881380141,http://nyti.ms/XvsHF2,https://www.nytimes.com/2013/03/09/nyregion/su...
...,...,...,...
15144,5281959998_10150485781054999,http://nyti.ms/1sVukYB,https://www.nytimes.com/2014/11/16/world/mexic...
15145,5281959998_10150485788419999,http://nyti.ms/1EO0Z8a,https://www.nytimes.com/2014/11/14/nyregion/sy...
15146,5281959998_10150485805269999,http://nyti.ms/11bNEth,https://www.nytimes.com/2014/11/16/business/an...
15147,5281959998_10150485814414999,http://nyti.ms/1uBLdf5,https://www.nytimes.com/2014/11/13/science/kee...


In [136]:
# import links expanded from messages
df_message_links = pd.read_json('data/expanded_links_frommessages.json', 
                                dtype='object')
df_message_links.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2217 entries, 1 to 3135
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             2217 non-null   object
 1   link_expanded  2217 non-null   object
dtypes: object(2)
memory usage: 52.0+ KB


In [20]:
# merge message links with main df
df = df.merge(df_message_links, how='left', left_on='id', right_on='id',
                   suffixes=('', '_new'))

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47867 entries, 0 to 47866
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 47867 non-null  object
 1   page_id            47867 non-null  int64 
 2   name               46574 non-null  object
 3   message            46132 non-null  object
 4   description        43512 non-null  object
 5   caption            42892 non-null  object
 6   post_type          47867 non-null  object
 7   status_type        47856 non-null  object
 8   likes_count        47867 non-null  int64 
 9   comments_count     47867 non-null  int64 
 10  shares_count       47867 non-null  int64 
 11  love_count         47867 non-null  int64 
 12  wow_count          47867 non-null  int64 
 13  haha_count         47867 non-null  int64 
 14  sad_count          47867 non-null  int64 
 15  thankful_count     47867 non-null  int64 
 16  angry_count        47867 non-null  int64

In [142]:
# import second batch of expanded links from original link field
df_expanded1a = pd.read_json('data/expanded_links1a.json', 
                                dtype='object')
df_expanded1a.head()

,0,1
0,5281959998_10150485819419999,https://cooking.nytimes.com/recipes/8245-smoke...
1,5281959998_10150485829769999,https://www.nytimes.com/2014/11/16/fashion/wed...
2,5281959998_10150485830749999,https://www.nytimes.com/2014/11/17/us/groups-i...
3,5281959998_10150485840494999,https://www.nytimes.com/2014/11/16/arts/televi...
4,5281959998_10150485850274999,https://www.nytimes.com/2014/11/15/us/one-drug...


In [143]:
df_expanded1a.rename(columns={0: 'id', 1: 'link_expanded'}, inplace=True)
df_expanded1a.head()

,id,link_expanded
0,5281959998_10150485819419999,https://cooking.nytimes.com/recipes/8245-smoke...
1,5281959998_10150485829769999,https://www.nytimes.com/2014/11/16/fashion/wed...
2,5281959998_10150485830749999,https://www.nytimes.com/2014/11/17/us/groups-i...
3,5281959998_10150485840494999,https://www.nytimes.com/2014/11/16/arts/televi...
4,5281959998_10150485850274999,https://www.nytimes.com/2014/11/15/us/one-drug...


In [108]:
df = df.merge(df_expanded1a, how='left', left_on='id', right_on='id',
                   suffixes=('', '_new'))
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47867 entries, 0 to 47866
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 47867 non-null  object 
 1   page_id            47867 non-null  int64  
 2   name               46574 non-null  object 
 3   message            46132 non-null  object 
 4   description        43512 non-null  object 
 5   caption            42892 non-null  object 
 6   post_type          47867 non-null  object 
 7   status_type        47856 non-null  object 
 8   likes_count        47867 non-null  int64  
 9   comments_count     47867 non-null  int64  
 10  shares_count       47867 non-null  int64  
 11  love_count         47867 non-null  int64  
 12  wow_count          47867 non-null  int64  
 13  haha_count         47867 non-null  int64  
 14  sad_count          47867 non-null  int64  
 15  thankful_count     47867 non-null  int64  
 16  angry_count        478